In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import re


# Assuming you have a way to load your vocab mapping (word to index)
# For simplicity, let's pretend we have a vocab dictionary and a reverse_vocab for encoding and decoding
vocab = {"[PAD]": 0, "[UNK]": 1}  # Add the rest of your vocabulary here
reverse_vocab = {v: k for k, v in vocab.items()}
pad_token_id = vocab["[PAD]"]
unk_token_id = vocab["[UNK]"]


class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.pattern = re.compile(r'[\w]+|[^\w\s]')  # Regex to split words and punctuation

    def encode(self, text):
        tokens = self.pattern.findall(text)  # Improved tokenization
        token_ids = []
        for token in tokens:
            subwords = self.find_subwords(token)
            token_ids.extend(subwords)
        return token_ids

    def find_subwords(self, token):
        subwords = []
        i = 0
        while i < len(token):
            found_subword = False
            for j in range(len(token), i, -1):
                subword = token[i:j]
                if subword in self.vocab:
                    subwords.append(self.vocab[subword])
                    i = j
                    found_subword = True
                    break
            if not found_subword:
                subwords.append(unk_token_id)  # Fallback to UNK
                i += 1  # Move to the next character
        return subwords

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoded_text = self.tokenizer.encode(text)

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        if padding_length > 0:
            encoded_text += [pad_token_id] * padding_length
        else:
            encoded_text = encoded_text[:self.max_seq_len]

        return torch.tensor(encoded_text, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Adjusting the EmbeddingLayer to not use the Tokenizer's non-existent vocab attribute
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    def forward(self, token_ids):
        return self.embedding(token_ids)

# Correcting TransformerEncoderLayer's forward method to properly use MultiHeadAttention
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, dropout=dropout)
        self.ffnn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.norm1(src)
        q, _ = self.self_attn(src2, src2, src2)
        src = src + self.dropout(q)
        src2 = self.norm2(src)
        src = src + self.dropout(self.ffnn(src2))
        return src

# Correction: Pooler squeezes the wrong dimension; it should squeeze dimension 0 (batch dimension is assumed to be 1 here)
class Pooler(nn.Module):
    def __init__(self, d_model):
        super(Pooler, self).__init__()
        self.linear = nn.Linear(d_model, d_model)
        self.activation = nn.Tanh()

    def forward(self, input_tensor):
        # Assuming input_tensor is of shape [batch_size, seq_len, d_model], take the first token's representations
        first_token_tensor = input_tensor[:, 0]
        pooled_output = self.linear(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

# Load pre-trained tokenizer and adjust vocab_size accordingly
tokenizer = Tokenizer(vocab=vocab)

# Assuming vocab_size is the length of your vocab dictionary
vocab_size = len(vocab)
embedding_dim = 128

# Define the model
model = nn.Sequential(
    EmbeddingLayer(vocab_size=vocab_size, embedding_dim=embedding_dim),
    TransformerEncoderLayer(d_model=embedding_dim, nhead=8, dim_feedforward=2048, dropout=0.1),
    Pooler(d_model=embedding_dim)
)

# Correcting the training and evaluation loop
# Load and preprocess data
train_data = pd.read_csv("train.csv")
train_texts = train_data["text"].tolist()
train_labels = train_data["label"].tolist()

# Convert texts and labels into a Dataset and DataLoader
max_seq_len = 512
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop corrected for proper input handling
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    total_loss = 0
    for token_ids, labels in train_dataloader:
        token_ids, labels = token_ids.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(token_ids)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")




'''
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_seq_len):
        super(EmbeddingLayer, self).__init__()
        self.token_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.positional_embeddings = PositionalEncoding(max_seq_len, embedding_dim)
        
    def forward(self, token_ids):
        token_embeds = self.token_embeddings(token_ids)  # [batch_size, seq_len, embedding_dim]
        position_embeds = self.positional_embeddings(token_embeds)  # [seq_len, embedding_dim]
        return token_embeds + position_embeds
'''




# v2

In [13]:
import os

# Print the current working directory
print("Current Working Directory:", os.getcwd())

os.chdir('D:\EXPERT_WEIGHTS')

print("Working Directory After change WD:", os.getcwd())


Current Working Directory: D:\EXPERT_WEIGHTS
Working Directory After change WD: D:\EXPERT_WEIGHTS


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import re
import collections
from collections import Counter
import math

def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def tokenize(text):
    return re.findall(r'\b\w+\b|[\s\.,!?;]', text)

def build_vocab(tokens, max_vocab_size=10000):
    token_freqs = Counter(tokens)
    sorted_tokens = sorted(token_freqs.items(), key=lambda x: (-x[1], x[0]))
    # Ensure special tokens are included
    vocab = {"[PAD]": 0, "[UNK]": 1, "[CLS]": 2, "[SEP]": 3, "[MASK]": 4}
    for token, _ in sorted_tokens[:max_vocab_size - len(vocab)]:
        vocab[token] = len(vocab)
    return vocab

def cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return 0.5 * (1. + math.cos(math.pi * progress)) 

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def combined_loss(output, target, model, l2_reg_strength=1.0, l1_reg_strength=0.0):
    task_loss = nn.CrossEntropyLoss()(output, target)  
    regularization_loss = 0

    for param in model.parameters():
        if isinstance(param, nn.Parameter):  
            regularization_loss += param.pow(2).sum() * l2_reg_strength  # L2
            regularization_loss += param.abs().sum() * l1_reg_strength  # L1

    return task_loss + regularization_loss

# Encoding Transformer Code

class AdaptiveDropoutLayer(nn.Module):
    def __init__(self, init_dropout_rate=0.1):
        super().__init__()
        self.log_alpha = nn.Parameter(torch.tensor(math.log(init_dropout_rate / (1 - init_dropout_rate))).float())  # Use logit transformation for stability

    def forward(self, x):
        p = torch.sigmoid(self.log_alpha) 
        return nn.functional.dropout(x, p=p, training=self.training) 

class AdaptiveWeightDecayOptimizer(optim.Optimizer):
    def __init__(self, params, lr=1e-3, init_l2_strength=0.01):
        super().__init__(params, {'lr': lr})
        self.log_l2_strength = nn.Parameter(torch.tensor(math.log(init_l2_strength)).float())

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = torch.exp(self.log_l2_strength)  
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay) 
                p.update(d_p, group['lr']) 

        return loss

class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, embedding_dim):
        super(PositionalEncoding, self).__init__()
        self.positional_embeddings = nn.Parameter(torch.zeros(max_seq_len, embedding_dim), requires_grad=False)
        
        position = torch.arange(0, max_seq_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * -(math.log(10000.0) / embedding_dim))
        
        self.positional_embeddings[:, 0::2] = torch.sin(position * div_term)
        self.positional_embeddings[:, 1::2] = torch.cos(position * div_term)

    def forward(self, x):
        return self.positional_embeddings[:x.size(1), :]

class MultiHeadLinformerAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, k=None):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.k = k if k is not None else embed_dim // num_heads  # Projection dimension

        # Separate projections for each head
        self.key_projections = nn.ModuleList([nn.Linear(embed_dim, self.k) for _ in range(num_heads)])
        self.value_projections = nn.ModuleList([nn.Linear(embed_dim, self.k) for _ in range(num_heads)]) 

        self.out_projection = nn.Linear(self.k * num_heads, embed_dim)

    def forward(self, query):
        seq_len, batch_size, _ = query.size()
        heads = []  # Store output from each head

        for head_idx in range(self.num_heads):
            projected_keys = self.key_projections[head_idx](query)
            projected_values = self.value_projections[head_idx](query)

            # Calculate attention using projected keys and values
            attention = torch.softmax(projected_keys.transpose(2, 3) @ projected_values, dim=-1) 

            out = attention @ projected_values.view(batch_size, seq_len, self.num_heads, self.k)
            out = out.transpose(1, 2).contiguous().view(seq_len, batch_size, self.embed_dim)

            heads.append(out)

        # Concatenate outputs from all heads
        out = torch.cat(heads, dim=-1) 
        out = self.out_projection(out) 
        return out

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer.encode(text)
        labels = encoded_text.copy()  # Copy encoded text for labels

        # Apply dynamic masking
        masked_indices = np.random.rand(len(encoded_text)) < 0.15
        for i in range(len(encoded_text)):
            if masked_indices[i]:
                encoded_text[i] = self.tokenizer.vocab["[MASK]"]

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        attention_mask = [1] * len(encoded_text) + [0] * padding_length
        encoded_text += [self.tokenizer.vocab["[PAD]"]] * padding_length
        labels += [-100] * padding_length  # Use -100 for padding positions, ignored by nn.CrossEntropyLoss

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }


class AdaptiveEmbeddingLayer(nn.Module):
    def __init__(self, vocab, freq_threshold, large_embed_dim, small_embed_dim, max_seq_len):
        super().__init__()
        self.split_vocab(vocab, freq_threshold)

        # Initialize embeddings with the correct size
        self.frequent_embeddings = nn.Embedding(num_embeddings=len(self.frequent_vocab), embedding_dim=large_embed_dim)
        self.infrequent_embeddings = nn.Embedding(num_embeddings=len(self.infrequent_vocab), embedding_dim=small_embed_dim)


        self.positional_embeddings = PositionalEncoding(max_seq_len, large_embed_dim)  


    def split_vocab(self, vocab, freq_threshold):
        token_counts = [(token, count) for token, count in vocab.items()] 
        token_counts.sort(key=lambda x: -x[1])  # Sort by frequency
        split_point = next(i for i, (_, count) in enumerate(token_counts) if count < freq_threshold)

        self.frequent_vocab = {token: i for i, (token, _) in enumerate(token_counts[:split_point])}
        self.infrequent_vocab = {token: i for i, (token, _) in enumerate(token_counts[split_point:])}

    def forward(self, token_ids):
        token_embeds = []
        for token_id in token_ids:
            if token_id in self.frequent_vocab:
                embed = self.frequent_embeddings(torch.tensor(token_id).long())
            else:
                embed = self.infrequent_embeddings(torch.tensor(token_id).long())
            token_embeds.append(embed)

        token_embeds = torch.stack(token_embeds)
        position_embeds = self.positional_embeddings(token_embeds)
        return token_embeds + position_embeds

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super(TransformerEncoderLayer, self).__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadLinformerAttention(embed_dim=d_model, num_heads=nhead)
        self.dropout1 = AdaptiveDropoutLayer()  # Use AdaptiveDropoutLayer
        self.norm2 = nn.LayerNorm(d_model)

        self.ffnn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            AdaptiveDropoutLayer(),  # Use AdaptiveDropoutLayer here as well
            nn.Linear(dim_feedforward, d_model),
        )
        self.dropout2 = AdaptiveDropoutLayer()  # And here

    def forward(self, src, src_mask=None):
        src2 = self.norm1(src)
        attn_output = self.attn(src2)
        src = src + self.dropout1(attn_output)
        src2 = self.norm2(src)
        src = src + self.dropout2(self.ffnn(src2))
        return src


class Pooler(nn.Module):
    def __init__(self, d_model):
        super(Pooler, self).__init__()
        self.linear = nn.Linear(d_model, d_model)
        self.activation = nn.Tanh()

    def forward(self, input_tensor):
        # Assuming input_tensor is of shape [batch_size, seq_len, d_model], take the first token's representations
        first_token_tensor = input_tensor[:, 0]
        pooled_output = self.linear(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_seq_len, nhead, dim_feedforward, 
                 freq_threshold, smaller_embed_dim):
        super(TransformerModel, self).__init__()
        self.embedding = AdaptiveEmbeddingLayer(
            vocab=vocab, 
            freq_threshold=freq_threshold,  
            large_embed_dim=embedding_dim,       
            small_embed_dim=smaller_embed_dim,   
            max_seq_len=max_seq_len
        )        
        self.encoder = TransformerEncoderLayer(embedding_dim, nhead, dim_feedforward)
        self.pooler = Pooler(embedding_dim)
    
    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        encoded = self.encoder(embedded, src_mask=attention_mask)
        pooled = self.pooler(encoded)
        return pooled

# Tokenizing Code

class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id

    def find_subwords(self, token):
        node = self.root
        subword_ids = []
        for char in token:
            if char in node.children:
                node = node.children[char]
                if node.token_id is not None:
                    subword_ids.append(node.token_id)
                    break  # Assuming one token maps to one subword for simplicity
            else:
                break  # No further subword match found
        if not subword_ids:  # If no subword was found
            subword_ids.append(self.unk_token_id if hasattr(self, 'unk_token_id') else 1) 
        return subword_ids
    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.

        Args:
            corpus: A text corpus represented as a list of strings.
        """

        self.vocab = self.init_vocab(corpus)

        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print(best)  # Track most frequent pair in each iteration

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.split()  # Assuming simple word splitting
            for word in words:
                vocab[word] += 1
        return vocab

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None 
        self.failure_pops = None

    def train(self, corpus):
        vocabulary = self._build_vocabulary(corpus)  # Build the word list
        self.trie, self.failure_links, self.failure_pops = self._precompute(vocabulary)

    def _encode(self, word):
        subword_ids = []
        current_node = self.trie.root

        for i, char in enumerate(word):
            if char in current_node.children:
                current_node = current_node.children[char]
                if current_node.token_id:
                    subword_ids.append(current_node.token_id)
            else:  # No direct match. Implement failure logic
                while True:
                    if current_node.failure_link is not None:
                        current_node = current_node.failure_link

                        # Check for pops to skip further backtracking
                        for _ in range(current_node.failure_pop):
                            current_node = current_node.failure_link

                        if char in current_node.children:
                            current_node = current_node.children[char]
                            if current_node.token_id:
                                subword_ids.append(current_node.token_id)
                            break  # Successful subword transition 

                    else:  # No failure link, we're back at root level
                        # Handle situation based on your application
                        # e.g., append an unknown token 
                        subword_ids.append(self.unk_token_id)  
                        break  

        return subword_ids

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        if model_type == "unigram":
            tokens = self._unigram_tokenize(corpus)  #  Hypothetical tokenize function
            vocabulary = self._build_unigram_vocab(tokens, vocab_size)
        elif model_type == "bpe":
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Step 1: Trie Construction
        self.trie = Trie()  # Assuming you have a Trie class as well
        for token in vocabulary:
            self.trie.insert(token)  

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root

    def _unigram_tokenize(self, corpus):
        """
        Tokenizes the given corpus into unigram tokens, checking against the built vocabulary.
        Tokens not found in the vocabulary are treated as unknowns.

        Args:
            corpus (str): The text corpus to tokenize.

        Returns:
            list of str: A list of tokens extracted from the corpus, adjusted to fit the vocabulary.
        """
        # Initial tokenization based on word boundaries and punctuation
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', corpus)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

class Tokenizer:
    def __init__(self, vocab, actual_vocab_size):
        self.vocab = vocab
        self.actual_vocab_size = actual_vocab_size
        self.unk_token_id = self.vocab.get("[UNK]", 1)  # Get ID of [UNK] 
        self.max_subword_length = max(len(token) for token in vocab.keys())
        self.pattern = re.compile(r'\b\w+\b|[\s\.,!?;]')

        # Build the Trie
        self.trie = Trie()
        for token, token_id in vocab.items():
            self.trie.insert(token, token_id)

    def _find_subwords(self, word):
        # 1. Trie Lookup 
        subword_ids = self.trie.find_subwords(word)

        # 2. Fallback to Original Logic 
        if len(subword_ids) == 1 and subword_ids[0] == self.unk_token_id:  
            subwords = []
            i = 0
            while i < len(word):
                for j in range(self.max_subword_length, 0, -1):
                    subword = word[i:i+j]
                    if subword in self.vocab:
                        subwords.append(self.vocab[subword])
                        i += j
                        break 
                else: 
                    subwords.append(self.vocab["[UNK]"])
                    i += 1
            subword_ids = subwords  # Replace with token IDs

        return subword_ids

    def encode(self, text):
        # Add [CLS] token at the beginning
        token_ids = [self.vocab["[CLS]"]]
        tokens = self.pattern.findall(text)
        for token in tokens:
            subword_ids = self._find_subwords(token)
            # Ensure all token IDs are within the actual vocabulary size
            subword_ids = [id if id < self.actual_vocab_size else self.vocab["[UNK]"] for id in subword_ids]
            token_ids.extend(subword_ids)
        # Add [SEP] token at the end
        token_ids.append(self.vocab["[SEP]"])
        return token_ids


# Training Code

# Load corpus and build vocab
corpus = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")
tokens = tokenize(corpus)
vocab = build_vocab(tokens)
actual_vocab_size = len(vocab)  # This includes [PAD], [UNK], [CLS], [SEP], [MASK]

print(f"Actual vocabulary size (including special tokens): {actual_vocab_size}")

tokenizer = Tokenizer(vocab=vocab, actual_vocab_size=actual_vocab_size)

# Data Loading (Illustrative)
train_texts = [corpus]  # Treat your whole sample as one "document"
train_dataset = TextDataset(train_texts, tokenizer, max_seq_len=512)

print("Tokenizer unk_token_id:", tokenizer.unk_token_id) 
print("Tokenizer Vocabulary:", tokenizer.vocab)

'''
# Sample text for testing your tokenizer
test_sentences = [
    "This is a sample sentence.",
    "Let's tokenize some unusual words with punctuation, shall we?",
    "1234 numbers or combinations? How does the tokenizer handle this?"
]

for sentence in test_sentences:
    tokenized_output = tokenizer.encode(sentence)
    print(f"Original Sentence: {sentence}")
    print(f"Tokenized Output: {tokenized_output}")
    print("-" * 50)  # A separator for visual clarity

'''

device='cpu'
freq_threshold_values = [10, 50, 100, 200, 500]  
best_validation_accuracy = 0.0 
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for freq_threshold in freq_threshold_values:

    # Model instantiation and training setup
    model = TransformerModel(
        vocab_size=actual_vocab_size,
        embedding_dim=128,
        max_seq_len=512,
        nhead=8,
        dim_feedforward=2048,
        freq_threshold=freq_threshold,  # Define your frequency threshold for splitting vocab
        smaller_embed_dim=64
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-4) 
    meta_optimizer = AdaptiveWeightDecayOptimizer(model.parameters(), lr=1e-5) 
    loss_fn = nn.CrossEntropyLoss()
    meta_update_freq = 5

    # Training loop corrected for model architecture
    for epoch in range(5):
        model.train()
        total_loss = 0

        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad() 
            input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            output = model(input_ids, attention_mask)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step() 

            # Meta-update occasionally 
            if (i + 1) % meta_update_freq == 0:
                meta_optimizer.zero_grad() 
                loss = combined_loss(output, labels, model) 
                loss.backward()
                meta_optimizer.step()  
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")

Actual vocabulary size (including special tokens): 1743
Tokenizer unk_token_id: 1
Tokenizer Vocabulary: {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, ' ': 5, '\n': 6, '.': 7, 'the': 8, ',': 9, 'of': 10, 'to': 11, 'and': 12, 'a': 13, 'is': 14, 'in': 15, 'that': 16, 'on': 17, '1': 18, 'networks': 19, 'shot': 20, 'for': 21, 'are': 22, 'we': 23, 'network': 24, 'We': 25, 'features': 26, 'dataset': 27, 'layer': 28, 'layers': 29, 'as': 30, '2': 31, 'from': 32, 'class': 33, 'with': 34, '0': 35, 'by': 36, 'each': 37, '5': 38, 'The': 39, 'classes': 40, 'training': 41, '3': 42, '4': 43, 'be': 44, 'performance': 45, 'or': 46, 'this': 47, 'learning': 48, 'trained': 49, 'A': 50, 'first': 51, 'points': 52, 'way': 53, 'distance': 54, 'task': 55, 'which': 56, 'embedding': 57, 'random': 58, 'set': 59, 'target': 60, 'an': 61, 'B': 62, 'al': 63, 'et': 64, 'prototypical': 65, '7': 66, 'can': 67, 'than': 68, 'few': 69, 'not': 70, 'at': 71, 'base': 72, 'two': 73, 'our': 74, 'results': 75, '6'

C:\Users\robbi\AppData\Local\Temp\ipykernel_35548\377544582.py:192: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embed = self.infrequent_embeddings(torch.tensor(token_id).long())


IndexError: index out of range in self

# V3

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import re
import collections
from collections import Counter
import math

def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def tokenize(text):
    return re.findall(r'\b\w+\b|[\s\.,!?;]', text)

def build_vocab(tokens, max_vocab_size=10000):
    token_freqs = Counter(tokens)
    sorted_tokens = sorted(token_freqs.items(), key=lambda x: (-x[1], x[0]))
    # Ensure special tokens are included
    vocab = {"[PAD]": 0, "[UNK]": 1, "[CLS]": 2, "[SEP]": 3, "[MASK]": 4}
    for token, _ in sorted_tokens[:max_vocab_size - len(vocab)]:
        vocab[token] = len(vocab)
    return vocab

def cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return 0.5 * (1. + math.cos(math.pi * progress)) 

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def combined_loss(output, target, model, l2_reg_strength=1.0, l1_reg_strength=0.0):
    task_loss = nn.CrossEntropyLoss()(output, target)  
    regularization_loss = 0

    for param in model.parameters():
        if isinstance(param, nn.Parameter):  
            regularization_loss += param.pow(2).sum() * l2_reg_strength  # L2
            regularization_loss += param.abs().sum() * l1_reg_strength  # L1

    return task_loss + regularization_loss


class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_seq_len= 512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer.encode(text)[:self.max_seq_len]         
        labels = encoded_text.copy()  # Copy encoded text for labels

        # Apply dynamic masking
        masked_indices = np.random.rand(len(encoded_text)) < 0.15
        for i in range(len(encoded_text)):
            if masked_indices[i]:
                encoded_text[i] = self.tokenizer.vocab["[MASK]"]

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        attention_mask = [1] * len(encoded_text) + [0] * padding_length
        encoded_text += [self.tokenizer.vocab["[PAD]"]] * padding_length
        labels += [-100] * padding_length  # Use -100 for padding positions

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }



# Encoding Transformer Code

class AdaptiveDropoutLayer(nn.Module):
    def __init__(self, init_dropout_rate=0.1):
        super(AdaptiveDropoutLayer, self).__init__()
        # Use logit transformation for stability
        self.log_alpha = nn.Parameter(torch.tensor(math.log(init_dropout_rate / (1 - init_dropout_rate))).float())

    def forward(self, x):
        p = torch.sigmoid(self.log_alpha)
        # Convert p from a tensor to a float
        p_value = p.item()  # This extracts the scalar value as a Python float
        return nn.functional.dropout(x, p=p_value, training=self.training)


class AdaptiveWeightDecayOptimizer(optim.Optimizer):
    def __init__(self, params, lr=1e-3, init_l2_strength=0.01):
        super().__init__(params, {'lr': lr})
        self.log_l2_strength = nn.Parameter(torch.tensor(math.log(init_l2_strength)).float())

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = torch.exp(self.log_l2_strength)  
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay) 
                p.update(d_p, group['lr']) 

        return loss

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len

        # Create positional encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add a batch dimension (B x T x C)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: Tensor of shape [Batch Size, Sequence Length, Embedding Dimension]
        # Adjust positional encoding to match the input size and device
        pe = self.pe[:, :x.size(1)]
        # Assuming x is on the correct device, pe will be automatically aligned to the same device
        return pe


class MultiHeadLinformerAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, k=None):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.k = k if k is not None else embed_dim // num_heads  # Projection dimension per head

        self.key_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.value_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.out_projection = nn.Linear(self.k * num_heads, embed_dim)

    def forward(self, query):
        batch_size, seq_len, _ = query.size()
        
        # Project keys and values
        keys = self.key_projections(query)
        values = self.value_projections(query)
        
        # Reshape into [batch_size, num_heads, seq_len, k]
        keys = keys.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        values = values.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        
        # Calculate attention (scaled dot-product attention)
        # Scaling by the square root of the depth of the key vectors to prevent large values in the dot product
        # which could push the softmax function into regions where it has extremely small gradients
        keys = keys / (self.k ** 0.5)
        attention_scores = torch.softmax(torch.matmul(keys, values.transpose(-2, -1)), dim=-1)
        
        # Apply attention to values
        out = torch.matmul(attention_scores, values)
        
        # Concatenate heads and project back to original embedding dimension
        out = out.transpose(1, 2).reshape(batch_size, seq_len, self.num_heads * self.k)
        out = self.out_projection(out)
        
        return out


class AdaptiveEmbeddingLayer(nn.Module):
    def __init__(self, vocab,  vocab_size, freq_threshold, large_embed_dim, small_embed_dim, max_seq_len):
        super(AdaptiveEmbeddingLayer, self).__init__()
        self.vocab = vocab
        self.vocab_size = vocab_size
        self.freq_threshold = freq_threshold
        self.large_embed_dim = large_embed_dim
        self.small_embed_dim = small_embed_dim
        self.max_seq_len = max_seq_len

        self.split_vocab(vocab, freq_threshold)  

        self.frequent_embeddings = nn.Embedding(num_embeddings=len(self.frequent_vocab), embedding_dim=large_embed_dim)
        self.infrequent_embeddings = nn.Embedding(num_embeddings=len(self.infrequent_vocab), embedding_dim=small_embed_dim)
        self.infrequent_projection = nn.Linear(small_embed_dim, large_embed_dim)
        self.positional_embeddings = PositionalEncoding(large_embed_dim, max_seq_len)


    def split_vocab(self, vocab, freq_threshold):
        token_counts = [(token, count) for token, count in vocab.items()]
        token_counts.sort(key=lambda x: -x[1])  # Sort by frequency
        split_point = next(i for i, (_, count) in enumerate(token_counts) if count < freq_threshold)
        
        self.frequent_vocab = dict(token_counts[:split_point])
        self.infrequent_vocab = dict(token_counts[split_point:])

    def forward(self, token_ids):
        device = token_ids.device
        seq_len = token_ids.size(1)
        batch_size = token_ids.size(0)  # Obtain batch size from token_ids tensor

        # Initialize embeddings tensor
        embeddings = torch.zeros(token_ids.shape[0], seq_len, self.large_embed_dim, device=device)

        # Map token_ids to indices for frequent and infrequent vocab
        frequent_indices = torch.zeros_like(token_ids)
        infrequent_indices = torch.zeros_like(token_ids)
        
        for token_id, index in self.vocab.items():
            mask = token_ids == token_id
            if token_id in self.frequent_vocab:
                # Map to index in frequent_vocab
                frequent_indices[mask] = self.frequent_vocab[token_id]
            elif token_id in self.infrequent_vocab:
                # Map to index in infrequent_vocab
                infrequent_indices[mask] = self.infrequent_vocab[token_id]

        # Create masks for frequent and infrequent tokens
        frequent_mask = frequent_indices > 0
        infrequent_mask = infrequent_indices > 0

        # Embed frequent tokens
        if frequent_mask.any():
            frequent_embeddings = self.frequent_embeddings(frequent_indices[frequent_mask])
            embeddings[frequent_mask] = frequent_embeddings

        # Embed and project infrequent tokens
        if infrequent_mask.any():
            infrequent_embeddings = self.infrequent_embeddings(infrequent_indices[infrequent_mask])
            infrequent_embeddings_projected = self.infrequent_projection(infrequent_embeddings)
            embeddings[infrequent_mask] = infrequent_embeddings_projected

        # Apply positional embeddings
        position_ids = torch.arange(0, seq_len, dtype=torch.long, device=device).unsqueeze(0)
        position_embeddings = self.positional_embeddings(position_ids)  # Generate for seq_len

        # Ensure positional embeddings are broadcastable to the embeddings tensor
        # This step may not be necessary if your positional embeddings are already correctly shaped
        if position_embeddings.size(0) != batch_size:
            position_embeddings = position_embeddings.expand(batch_size, -1, -1)

        print(f"Embeddings shape: {embeddings.shape}")
        print(f"Positional embeddings shape: {position_embeddings.shape}")
        embeddings += position_embeddings

        return embeddings



class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super(TransformerEncoderLayer, self).__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadLinformerAttention(embed_dim=d_model, num_heads=nhead)
        self.dropout1 = AdaptiveDropoutLayer()  # Use AdaptiveDropoutLayer
        self.norm2 = nn.LayerNorm(d_model)

        self.ffnn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            AdaptiveDropoutLayer(),  # Use AdaptiveDropoutLayer here as well
            nn.Linear(dim_feedforward, d_model),
        )
        self.dropout2 = AdaptiveDropoutLayer()  # And here
 
    def forward(self, src, src_mask=None):
        src2 = self.norm1(src)
        attn_output = self.attn(src2)
        src = src + self.dropout1(attn_output)
        src2 = self.norm2(src)
        src = src + self.dropout2(self.ffnn(src2))
        return src


class Pooler(nn.Module):
    def __init__(self, d_model):
        super(Pooler, self).__init__()
        self.linear = nn.Linear(d_model, d_model)
        self.activation = nn.Tanh()

    def forward(self, input_tensor):
        # Assuming input_tensor is of shape [batch_size, seq_len, d_model], take the first token's representations
        first_token_tensor = input_tensor[:, 0]
        pooled_output = self.linear(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class TransformerModel(nn.Module):
    def __init__(self, vocab, vocab_size, embedding_dim, max_seq_len, nhead, dim_feedforward, 
                 freq_threshold, smaller_embed_dim):
        super(TransformerModel, self).__init__()
        self.embedding = AdaptiveEmbeddingLayer(
            vocab=vocab,
            vocab_size=vocab_size, 
            freq_threshold=freq_threshold,  
            large_embed_dim=embedding_dim,       
            small_embed_dim=smaller_embed_dim,   
            max_seq_len=max_seq_len
        )
        self.encoder = TransformerEncoderLayer(embedding_dim, nhead, dim_feedforward)
        self.pooler = Pooler(embedding_dim)  # Retain Pooler for sentence-level representation
        # Add an output projection layer for token-level predictions
        self.output_projection = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        encoded = self.encoder(embedded, src_mask=attention_mask)
        # Get pooled output for sentence-level tasks
        pooled_output = self.pooler(encoded)
        # Project encoded output to vocabulary size for token-level predictions
        logits = self.output_projection(encoded)
        return logits, pooled_output

# Tokenizing Code

class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id

    def find_subwords(self, token):
        node = self.root
        subword_ids = []
        for char in token:
            if char in node.children:
                node = node.children[char]
                if node.token_id is not None:
                    subword_ids.append(node.token_id)
                    break  # Assuming one token maps to one subword for simplicity
            else:
                break  # No further subword match found
        if not subword_ids:  # If no subword was found
            subword_ids.append(self.unk_token_id if hasattr(self, 'unk_token_id') else 1) 
        return subword_ids
    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.

        Args:
            corpus: A text corpus represented as a list of strings.
        """

        self.vocab = self.init_vocab(corpus)

        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print(best)  # Track most frequent pair in each iteration

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.split()  # Assuming simple word splitting
            for word in words:
                vocab[word] += 1
        return vocab

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None 
        self.failure_pops = None

    def train(self, corpus):
        vocabulary = self._build_vocabulary(corpus)  # Build the word list
        self.trie, self.failure_links, self.failure_pops = self._precompute(vocabulary)

    def _encode(self, word):
        subword_ids = []
        current_node = self.trie.root

        for i, char in enumerate(word):
            if char in current_node.children:
                current_node = current_node.children[char]
                if current_node.token_id:
                    subword_ids.append(current_node.token_id)
            else:  # No direct match. Implement failure logic
                while True:
                    if current_node.failure_link is not None:
                        current_node = current_node.failure_link

                        # Check for pops to skip further backtracking
                        for _ in range(current_node.failure_pop):
                            current_node = current_node.failure_link

                        if char in current_node.children:
                            current_node = current_node.children[char]
                            if current_node.token_id:
                                subword_ids.append(current_node.token_id)
                            break  # Successful subword transition 

                    else:  # No failure link, we're back at root level
                        # Handle situation based on your application
                        # e.g., append an unknown token 
                        subword_ids.append(self.unk_token_id)  
                        break  

        return subword_ids

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        if model_type == "unigram":
            tokens = self._unigram_tokenize(corpus)  #  Hypothetical tokenize function
            vocabulary = self._build_unigram_vocab(tokens, vocab_size)
        elif model_type == "bpe":
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Step 1: Trie Construction
        self.trie = Trie()  # Assuming you have a Trie class as well
        for token in vocabulary:
            self.trie.insert(token)  

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root

    def _unigram_tokenize(self, corpus):
        """
        Tokenizes the given corpus into unigram tokens, checking against the built vocabulary.
        Tokens not found in the vocabulary are treated as unknowns.

        Args:
            corpus (str): The text corpus to tokenize.

        Returns:
            list of str: A list of tokens extracted from the corpus, adjusted to fit the vocabulary.
        """
        # Initial tokenization based on word boundaries and punctuation
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', corpus)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

class Tokenizer:
    def __init__(self, vocab, actual_vocab_size):
        self.vocab = vocab
        self.actual_vocab_size = actual_vocab_size
        self.unk_token_id = self.vocab.get("[UNK]", 1)  # Get ID of [UNK] 
        self.max_subword_length = max(len(token) for token in vocab.keys())
        self.pattern = re.compile(r'\b\w+\b|[\s\.,!?;]')

        # Build the Trie
        self.trie = Trie()
        for token, token_id in vocab.items():
            self.trie.insert(token, token_id)

    def _find_subwords(self, word):
        # 1. Trie Lookup 
        subword_ids = self.trie.find_subwords(word)

        # 2. Fallback to Original Logic 
        if len(subword_ids) == 1 and subword_ids[0] == self.unk_token_id:  
            subwords = []
            i = 0
            while i < len(word):
                for j in range(self.max_subword_length, 0, -1):
                    subword = word[i:i+j]
                    if subword in self.vocab:
                        subwords.append(self.vocab[subword])
                        i += j
                        break 
                else: 
                    subwords.append(self.vocab["[UNK]"])
                    i += 1
            subword_ids = subwords  # Replace with token IDs

        return subword_ids

    def encode(self, text):
        token_ids = [self.vocab.get("[CLS]", 1)]  # Use [CLS] token or [UNK] if not found
        tokens = self.pattern.findall(text)
        for token in tokens:
            token_id = self.vocab.get(token, self.vocab.get("[UNK]", 1))  # Fallback to [UNK] if token is not found
            token_ids.append(token_id)
        token_ids.append(self.vocab.get("[SEP]", 1))  # Use [SEP] token or [UNK] if not found
        return token_ids


# Training Code

# Load corpus and build vocab
corpus = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")
tokens = tokenize(corpus)
vocab = build_vocab(tokens)
actual_vocab_size = len(vocab)  # This includes [PAD], [UNK], [CLS], [SEP], [MASK]

print(f"Actual vocabulary size (including special tokens): {actual_vocab_size}")

tokenizer = Tokenizer(vocab=vocab, actual_vocab_size=actual_vocab_size)

# Data Loading (Illustrative)
train_texts = [corpus]  # Treat your whole sample as one "document"
train_dataset = TextDataset(train_texts, tokenizer, max_seq_len=512)

print("Tokenizer unk_token_id:", tokenizer.unk_token_id) 
print("Tokenizer Vocabulary:", tokenizer.vocab)



device='cpu'
freq_threshold_values = [10, 50, 100, 200, 500]  
best_validation_accuracy = 0.0 
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for freq_threshold in freq_threshold_values:

    # Model instantiation and training setup
    model = TransformerModel(
        vocab = vocab,
        vocab_size=actual_vocab_size,     
        embedding_dim=128,
        max_seq_len=512,
        nhead=8,
        dim_feedforward=2048,
        freq_threshold=freq_threshold,  # frequency threshold for splitting vocab
        smaller_embed_dim=64
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-4) 
    meta_optimizer = AdaptiveWeightDecayOptimizer(model.parameters(), lr=1e-5) 
    loss_fn = nn.CrossEntropyLoss()
    meta_update_freq = 5

    # Training loop adjusted for the updated model architecture
    for epoch in range(5):
        model.train()
        total_loss = 0

        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Assuming labels are of shape [batch_size, sequence_length]

            # Forward pass, model now returns logits and pooled_output
            logits, pooled_output = model(input_ids, attention_mask)
            
            # Correctly reshape logits to match the labels' shape
            # Change from [1, 512, vocab_size] to [512, vocab_size] to align with labels
            logits = logits.view(-1, logits.size(-1))  # Reshape logits for loss calculation
            
            labels = labels.view(-1)  # Ensure labels are a flat vector

            # Calculate loss using logits for token-level predictions
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Meta-update occasionally
            if (i + 1) % meta_update_freq == 0:
                meta_optimizer.zero_grad()
                # Recalculate or reuse the loss for the meta-update
                meta_loss = combined_loss(logits.detach(), labels.detach(), model)
                meta_loss.backward()
                meta_optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")


# Save model state
model_path = "D:\\EXPERT_WEIGHTS\\encoding_transformer.bin"
torch.save(model.state_dict(), model_path)

# Save tokenizer using pickle for simplicity
import pickle
tokenizer_path = "D:\\EXPERT_WEIGHTS\\tokenizer.pkl"
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)


# Assuming TransformerModel and Tokenizer classes are defined in the scope

# Load the model
model_loaded = TransformerModel(
    vocab=vocab,  # Ensure `vocab` is loaded or defined in the scope
    vocab_size=actual_vocab_size,
    embedding_dim=128,
    max_seq_len=512,
    nhead=8,
    dim_feedforward=2048,
    freq_threshold=freq_threshold,  # Define or load `freq_threshold` as appropriate
    smaller_embed_dim=64
)
model_loaded.load_state_dict(torch.load("D:\\EXPERT_WEIGHTS\\encoding_transformer.bin"))
model_loaded.eval()  # Set to evaluation mode

# Load the tokenizer
with open("D:\\EXPERT_WEIGHTS\\tokenizer.pkl", 'rb') as f:
    tokenizer_loaded = pickle.load(f)


# Example text
text = "Here is some text to encode"

# Tokenize the input
encoded_input = tokenizer_loaded.encode(text, add_special_tokens=True, return_tensors="pt")
# Predict with your model
with torch.no_grad():  # No need to calculate gradients
    output = model_loaded(encoded_input)

# Process the output as needed


Actual vocabulary size (including special tokens): 1743
Tokenizer unk_token_id: 1
Tokenizer Vocabulary: {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, ' ': 5, '\n': 6, '.': 7, 'the': 8, ',': 9, 'of': 10, 'to': 11, 'and': 12, 'a': 13, 'is': 14, 'in': 15, 'that': 16, 'on': 17, '1': 18, 'networks': 19, 'shot': 20, 'for': 21, 'are': 22, 'we': 23, 'network': 24, 'We': 25, 'features': 26, 'dataset': 27, 'layer': 28, 'layers': 29, 'as': 30, '2': 31, 'from': 32, 'class': 33, 'with': 34, '0': 35, 'by': 36, 'each': 37, '5': 38, 'The': 39, 'classes': 40, 'training': 41, '3': 42, '4': 43, 'be': 44, 'performance': 45, 'or': 46, 'this': 47, 'learning': 48, 'trained': 49, 'A': 50, 'first': 51, 'points': 52, 'way': 53, 'distance': 54, 'task': 55, 'which': 56, 'embedding': 57, 'random': 58, 'set': 59, 'target': 60, 'an': 61, 'B': 62, 'al': 63, 'et': 64, 'prototypical': 65, '7': 66, 'can': 67, 'than': 68, 'few': 69, 'not': 70, 'at': 71, 'base': 72, 'two': 73, 'our': 74, 'results': 75, '6'

# SP and BPE Hybrid Tokenizer

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import re
import collections
from collections import Counter
import math

def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def tokenize(text):
    return re.findall(r'\b\w+\b|[\s\.,!?;]', text)

def build_vocab(tokens, max_vocab_size=10000):
    token_freqs = Counter(tokens)
    sorted_tokens = sorted(token_freqs.items(), key=lambda x: (-x[1], x[0]))
    # Ensure special tokens are included
    vocab = {"[PAD]": 0, "[UNK]": 1, "[CLS]": 2, "[SEP]": 3, "[MASK]": 4}
    for token, _ in sorted_tokens[:max_vocab_size - len(vocab)]:
        vocab[token] = len(vocab)
    return vocab

def cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return 0.5 * (1. + math.cos(math.pi * progress)) 

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def combined_loss(output, target, model, l2_reg_strength=1.0, l1_reg_strength=0.0):
    task_loss = nn.CrossEntropyLoss()(output, target)  
    regularization_loss = 0

    for param in model.parameters():
        if isinstance(param, nn.Parameter):  
            regularization_loss += param.pow(2).sum() * l2_reg_strength  # L2
            regularization_loss += param.abs().sum() * l1_reg_strength  # L1

    return task_loss + regularization_loss

# Encoding Transformer Code

class AdaptiveDropoutLayer(nn.Module):
    def __init__(self, init_dropout_rate=0.1):
        super(AdaptiveDropoutLayer, self).__init__()
        # Use logit transformation for stability
        self.log_alpha = nn.Parameter(torch.tensor(math.log(init_dropout_rate / (1 - init_dropout_rate))).float())

    def forward(self, x):
        p = torch.sigmoid(self.log_alpha)
        # Convert p from a tensor to a float
        p_value = p.item()  # This extracts the scalar value as a Python float
        return nn.functional.dropout(x, p=p_value, training=self.training)


class AdaptiveWeightDecayOptimizer(optim.Optimizer):
    def __init__(self, params, lr=1e-3, init_l2_strength=0.01):
        super().__init__(params, {'lr': lr})
        self.log_l2_strength = nn.Parameter(torch.tensor(math.log(init_l2_strength)).float())

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = torch.exp(self.log_l2_strength)  
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay) 
                p.update(d_p, group['lr']) 

        return loss

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len

        # Create positional encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add a batch dimension (B x T x C)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: Tensor of shape [Batch Size, Sequence Length, Embedding Dimension]
        # Adjust positional encoding to match the input size and device
        pe = self.pe[:, :x.size(1)]
        # Assuming x is on the correct device, pe will be automatically aligned to the same device
        return pe


class MultiHeadLinformerAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, k=None):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.k = k if k is not None else embed_dim // num_heads  # Projection dimension per head

        self.key_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.value_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.out_projection = nn.Linear(self.k * num_heads, embed_dim)

    def forward(self, query):
        batch_size, seq_len, _ = query.size()
        
        # Project keys and values
        keys = self.key_projections(query)
        values = self.value_projections(query)
        
        # Reshape into [batch_size, num_heads, seq_len, k]
        keys = keys.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        values = values.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        
        # Calculate attention (scaled dot-product attention)
        # Scaling by the square root of the depth of the key vectors to prevent large values in the dot product
        # which could push the softmax function into regions where it has extremely small gradients
        keys = keys / (self.k ** 0.5)
        attention_scores = torch.softmax(torch.matmul(keys, values.transpose(-2, -1)), dim=-1)
        
        # Apply attention to values
        out = torch.matmul(attention_scores, values)
        
        # Concatenate heads and project back to original embedding dimension
        out = out.transpose(1, 2).reshape(batch_size, seq_len, self.num_heads * self.k)
        out = self.out_projection(out)
        
        return out





class AdaptiveEmbeddingLayer(nn.Module):
    def __init__(self, vocab,  vocab_size, freq_threshold, large_embed_dim, small_embed_dim, max_seq_len):
        super(AdaptiveEmbeddingLayer, self).__init__()
        self.vocab = vocab
        self.vocab_size = vocab_size
        self.freq_threshold = freq_threshold
        self.large_embed_dim = large_embed_dim
        self.small_embed_dim = small_embed_dim
        self.max_seq_len = max_seq_len

        self.split_vocab(vocab, freq_threshold)  

        self.frequent_embeddings = nn.Embedding(num_embeddings=len(self.frequent_vocab), embedding_dim=large_embed_dim)
        self.infrequent_embeddings = nn.Embedding(num_embeddings=len(self.infrequent_vocab), embedding_dim=small_embed_dim)
        self.infrequent_projection = nn.Linear(small_embed_dim, large_embed_dim)
        self.positional_embeddings = PositionalEncoding(large_embed_dim, max_seq_len)


    def split_vocab(self, vocab, freq_threshold):
        token_counts = [(token, count) for token, count in vocab.items()]
        token_counts.sort(key=lambda x: -x[1])  # Sort by frequency
        split_point = next(i for i, (_, count) in enumerate(token_counts) if count < freq_threshold)
        
        self.frequent_vocab = dict(token_counts[:split_point])
        self.infrequent_vocab = dict(token_counts[split_point:])

    def forward(self, token_ids):
        device = token_ids.device
        seq_len = token_ids.size(1)
        batch_size = token_ids.size(0)  # Obtain batch size from token_ids tensor

        # Initialize embeddings tensor
        embeddings = torch.zeros(token_ids.shape[0], seq_len, self.large_embed_dim, device=device)

        # Map token_ids to indices for frequent and infrequent vocab
        frequent_indices = torch.zeros_like(token_ids)
        infrequent_indices = torch.zeros_like(token_ids)
        
        for token_id, index in self.vocab.items():
            mask = token_ids == token_id
            if token_id in self.frequent_vocab:
                # Map to index in frequent_vocab
                frequent_indices[mask] = self.frequent_vocab[token_id]
            elif token_id in self.infrequent_vocab:
                # Map to index in infrequent_vocab
                infrequent_indices[mask] = self.infrequent_vocab[token_id]

        # Create masks for frequent and infrequent tokens
        frequent_mask = frequent_indices > 0
        infrequent_mask = infrequent_indices > 0

        # Embed frequent tokens
        if frequent_mask.any():
            frequent_embeddings = self.frequent_embeddings(frequent_indices[frequent_mask])
            embeddings[frequent_mask] = frequent_embeddings

        # Embed and project infrequent tokens
        if infrequent_mask.any():
            infrequent_embeddings = self.infrequent_embeddings(infrequent_indices[infrequent_mask])
            infrequent_embeddings_projected = self.infrequent_projection(infrequent_embeddings)
            embeddings[infrequent_mask] = infrequent_embeddings_projected

        # Apply positional embeddings
        position_ids = torch.arange(0, seq_len, dtype=torch.long, device=device).unsqueeze(0)
        position_embeddings = self.positional_embeddings(position_ids)  # Generate for seq_len

        # Ensure positional embeddings are broadcastable to the embeddings tensor
        # This step may not be necessary if your positional embeddings are already correctly shaped
        if position_embeddings.size(0) != batch_size:
            position_embeddings = position_embeddings.expand(batch_size, -1, -1)

        print(f"Embeddings shape: {embeddings.shape}")
        print(f"Positional embeddings shape: {position_embeddings.shape}")
        embeddings += position_embeddings

        return embeddings



class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super(TransformerEncoderLayer, self).__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadLinformerAttention(embed_dim=d_model, num_heads=nhead)
        self.dropout1 = AdaptiveDropoutLayer()  # Use AdaptiveDropoutLayer
        self.norm2 = nn.LayerNorm(d_model)

        self.ffnn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            AdaptiveDropoutLayer(),  # Use AdaptiveDropoutLayer here as well
            nn.Linear(dim_feedforward, d_model),
        )
        self.dropout2 = AdaptiveDropoutLayer()  # And here
 
    def forward(self, src, src_mask=None):
        src2 = self.norm1(src)
        attn_output = self.attn(src2)
        src = src + self.dropout1(attn_output)
        src2 = self.norm2(src)
        src = src + self.dropout2(self.ffnn(src2))
        return src


class Pooler(nn.Module):
    def __init__(self, d_model):
        super(Pooler, self).__init__()
        self.linear = nn.Linear(d_model, d_model)
        self.activation = nn.Tanh()

    def forward(self, input_tensor):
        # Assuming input_tensor is of shape [batch_size, seq_len, d_model], take the first token's representations
        first_token_tensor = input_tensor[:, 0]
        pooled_output = self.linear(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class TransformerModel(nn.Module):
    def __init__(self, vocab, vocab_size, embedding_dim, max_seq_len, nhead, dim_feedforward, 
                 freq_threshold, smaller_embed_dim):
        super(TransformerModel, self).__init__()
        self.embedding = AdaptiveEmbeddingLayer(
            vocab=vocab,
            vocab_size=vocab_size, 
            freq_threshold=freq_threshold,  
            large_embed_dim=embedding_dim,       
            small_embed_dim=smaller_embed_dim,   
            max_seq_len=max_seq_len
        )
        self.encoder = TransformerEncoderLayer(embedding_dim, nhead, dim_feedforward)
        self.pooler = Pooler(embedding_dim)  # Retain Pooler for sentence-level representation
        # Add an output projection layer for token-level predictions
        self.output_projection = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        encoded = self.encoder(embedded, src_mask=attention_mask)
        # Get pooled output for sentence-level tasks
        pooled_output = self.pooler(encoded)
        # Project encoded output to vocabulary size for token-level predictions
        logits = self.output_projection(encoded)
        return logits, pooled_output

# Tokenizing Code
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_seq_len= 512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer.encode(text)[:self.max_seq_len]         
        labels = encoded_text.copy()  # Copy encoded text for labels

        # Apply dynamic masking
        masked_indices = np.random.rand(len(encoded_text)) < 0.15
        for i in range(len(encoded_text)):
            if masked_indices[i]:
                encoded_text[i] = self.tokenizer.vocab["[MASK]"]

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        attention_mask = [1] * len(encoded_text) + [0] * padding_length
        encoded_text += [self.tokenizer.vocab["[PAD]"]] * padding_length
        labels += [-100] * padding_length  # Use -100 for padding positions

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }
    
class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup
        self.frequency = 0  # Track frequency of subwords for simplified ranking

from heapq import nlargest

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency  # Assign frequency at the leaf node


    def find_subwords(self, token):
        """
        Finds the most probable subwords for a given token based on frequency.
        """
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Convert frequency to a simple probability for demonstration
                probability = current_node.frequency / sum(node.frequency for node in collected_subwords)
                collected_subwords.append((subword, probability, current_node.token_id))

            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, collected_subwords)

        dfs(node)
        # Instead of selecting based purely on frequency, consider the 'probability'
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword[2] for subword in best_subwords][:5] if best_subwords else [self.unk_token_id]

    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.

        Args:
            corpus: A text corpus represented as a list of strings.
        """

        self.vocab = self.init_vocab(corpus)

        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print(best)  # Track most frequent pair in each iteration

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.split()  # Assuming simple word splitting
            for word in words:
                vocab[word] += 1
        return vocab

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None 
        self.failure_pops = None

    def train(self, corpus):
        vocabulary = self._build_vocabulary(corpus)  # Build the word list
        self.trie, self.failure_links, self.failure_pops = self._precompute(vocabulary)

    def _encode(self, text):
        """
        Tokenizes input text into subwords, using a trie to find the most probable subword sequences.
        """
        tokens = text.split()  # Simplified tokenization
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        if model_type == "unigram":
            tokens = self._unigram_tokenize(corpus)  #  Hypothetical tokenize function
            vocabulary = self._build_unigram_vocab(tokens, vocab_size)
        elif model_type == "bpe":
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Step 1: Trie Construction
        self.trie = Trie()  # Assuming you have a Trie class as well
        for token in vocabulary:
            self.trie.insert(token)  

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root

    def _unigram_tokenize(self, corpus):
        """
        Tokenizes the given corpus into unigram tokens, checking against the built vocabulary.
        Tokens not found in the vocabulary are treated as unknowns.

        Args:
            corpus (str): The text corpus to tokenize.

        Returns:
            list of str: A list of tokens extracted from the corpus, adjusted to fit the vocabulary.
        """
        # Initial tokenization based on word boundaries and punctuation
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', corpus)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens


class HybridTokenizer:
    def __init__(self, bpe_model, sentence_piece_model, vocab):
        self.bpe_model = bpe_model
        self.sentence_piece_model = sentence_piece_model
        self.vocab = vocab  # Your existing vocabulary

    def encode(self, text):
        # Apply BPE tokenization
        bpe_tokens = self.bpe_model.encode(text)
        # Further tokenize each BPE token with SentencePiece
        sp_tokens = [self.sentence_piece_model._encode(token) for token in bpe_tokens]
        # Flatten the list of lists
        flat_tokens = [item for sublist in sp_tokens for item in sublist]
        # Convert flat tokens to token IDs
        token_ids = [self.vocab.get(token, self.vocab.get("[UNK]", 1)) for token in flat_tokens]
        return token_ids


class Tokenizer:
    def __init__(self, vocab, actual_vocab_size):
        self.vocab = vocab
        self.actual_vocab_size = actual_vocab_size
        self.unk_token_id = self.vocab.get("[UNK]", 1)  # Get ID of [UNK] 
        self.max_subword_length = max(len(token) for token in vocab.keys())
        self.pattern = re.compile(r'\b\w+\b|[\s\.,!?;]')

        # Build the Trie
        self.trie = Trie()
        for token, token_id in vocab.items():
            self.trie.insert(token, token_id)

    def _find_subwords(self, word):
        # 1. Trie Lookup 
        subword_ids = self.trie.find_subwords(word)

        # 2. Fallback to Original Logic 
        if len(subword_ids) == 1 and subword_ids[0] == self.unk_token_id:  
            subwords = []
            i = 0
            while i < len(word):
                for j in range(self.max_subword_length, 0, -1):
                    subword = word[i:i+j]
                    if subword in self.vocab:
                        subwords.append(self.vocab[subword])
                        i += j
                        break 
                else: 
                    subwords.append(self.vocab["[UNK]"])
                    i += 1
            subword_ids = subwords  # Replace with token IDs

        return subword_ids

    def encode(self, text):
        token_ids = [self.vocab.get("[CLS]", 1)]  # Use [CLS] token or [UNK] if not found
        tokens = self.pattern.findall(text)
        for token in tokens:
            token_id = self.vocab.get(token, self.vocab.get("[UNK]", 1))  # Fallback to [UNK] if token is not found
            token_ids.append(token_id)
        token_ids.append(self.vocab.get("[SEP]", 1))  # Use [SEP] token or [UNK] if not found
        return token_ids



class HybridTokenizer_v2:
    def __init__(self, sentence_piece_model, bpe_model):
        # Load or initialize the SentencePiece and BPE models
        self.sentence_piece_model = sentence_piece_model  # Assume this is a loaded or initialized SentencePiece model
        self.bpe_model = bpe_model  # Assume this is a loaded or initialized BPE model

    def encode(self, text):
        # Encode text with SentencePiece
        sp_tokens = self.sentence_piece_model.encode(text, out_type=str)

        # Further encode each SentencePiece token with BPE
        tokens = []
        for token in sp_tokens:
            tokens.extend(self.bpe_model.encode(token))

        # Convert tokens to token IDs based on BPE's vocabulary
        token_ids = [self.bpe_model.token_to_id(token) for token in tokens]
        return token_ids

    def decode(self, token_ids):
        # Convert token IDs back to tokens based on BPE's vocabulary
        tokens = [self.bpe_model.id_to_token(token_id) for token_id in token_ids]

        # Decode tokens back to text
        text = self.sentence_piece_model.decode(tokens)
        return text

    def token_to_id(self, token):
        # Method to convert a token to its ID using BPE vocabulary
        return self.bpe_model.token_to_id(token)

    def id_to_token(self, token_id):
        # Method to convert a token ID back to its token using BPE vocabulary
        return self.bpe_model.id_to_token(token_id)


# Training Code

# Load corpus and build vocab
corpus = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")
tokens = tokenize(corpus)
vocab = build_vocab(tokens)
actual_vocab_size = len(vocab)  # This includes [PAD], [UNK], [CLS], [SEP], [MASK]
train_texts = [corpus]  # Treat your whole sample as one "document"
print(f"Actual vocabulary size (including special tokens): {actual_vocab_size}")

# Regular Tokenizer
#tokenizer = Tokenizer(vocab=vocab, actual_vocab_size=actual_vocab_size)
#train_dataset = TextDataset(train_texts, tokenizer, max_seq_len=512)
#train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Hybrid option
bpe_model = BPE()
bpe_model.vocab = vocab
bpe_model.train(corpus)  # Train BPE with your corpus
sentence_piece_model = SentencePiece()
hybrid_tokenizer = HybridTokenizer(bpe_model, sentence_piece_model, vocab)
hybrid_train_dataset = TextDataset(train_texts, hybrid_tokenizer, max_seq_len=512)
hybrid_train_dataloader = DataLoader(hybrid_train_dataset, batch_size=32, shuffle=True)


device='cpu'
freq_threshold_values = [10, 50, 100, 200, 500]  
best_validation_accuracy = 0.0 

for freq_threshold in freq_threshold_values:

    # Model instantiation and training setup
    model = TransformerModel(
        vocab = vocab,
        vocab_size=actual_vocab_size,     
        embedding_dim=128,
        max_seq_len=512,
        nhead=8,
        dim_feedforward=2048,
        freq_threshold=freq_threshold,  # frequency threshold for splitting vocab
        smaller_embed_dim=64
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-4) 
    meta_optimizer = AdaptiveWeightDecayOptimizer(model.parameters(), lr=1e-5) 
    loss_fn = nn.CrossEntropyLoss()
    meta_update_freq = 5

    # Training loop adjusted for the updated model architecture
    for epoch in range(5):
        model.train()
        total_loss = 0

        for i, batch in enumerate(hybrid_train_dataloader):
            optimizer.zero_grad()
            input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Assuming labels are of shape [batch_size, sequence_length]

            # Forward pass, model now returns logits and pooled_output
            logits, pooled_output = model(input_ids, attention_mask)
            
            # Correctly reshape logits to match the labels' shape
            # Change from [1, 512, vocab_size] to [512, vocab_size] to align with labels
            logits = logits.view(-1, logits.size(-1))  # Reshape logits for loss calculation
            
            labels = labels.view(-1)  # Ensure labels are a flat vector

            # Calculate loss using logits for token-level predictions
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Meta-update occasionally
            if (i + 1) % meta_update_freq == 0:
                meta_optimizer.zero_grad()
                # Recalculate or reuse the loss for the meta-update
                meta_loss = combined_loss(logits.detach(), labels.detach(), model)
                meta_loss.backward()
                meta_optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {total_loss / len(hybrid_train_dataloader)}")



Actual vocabulary size (including special tokens): 1743


ValueError: max() arg is an empty sequence

In [ ]:
# Save model state
model_path = "D:\\EXPERT_WEIGHTS\\encoding_transformer.bin"
torch.save(model.state_dict(), model_path)

# Save tokenizer using pickle for simplicity
import pickle
tokenizer_path = "D:\\EXPERT_WEIGHTS\\tokenizer.pkl"
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)


# Load the model
model_loaded = TransformerModel(
    vocab=vocab,  # Ensure `vocab` is loaded or defined in the scope
    vocab_size=actual_vocab_size,
    embedding_dim=128,
    max_seq_len=512,
    nhead=8,
    dim_feedforward=2048,
    freq_threshold=freq_threshold,  # Define or load `freq_threshold` as appropriate
    smaller_embed_dim=64
)
model_loaded.load_state_dict(torch.load("D:\\EXPERT_WEIGHTS\\encoding_transformer.bin"))
model_loaded.eval()  # Set to evaluation mode

# Load the tokenizer
with open("D:\\EXPERT_WEIGHTS\\tokenizer.pkl", 'rb') as f:
    tokenizer_loaded = pickle.load(f)


# Example text
text = "Here is some text to encode"

# Tokenize the input
encoded_input = tokenizer_loaded.encode(text, add_special_tokens=True, return_tensors="pt")
# Predict with your model
with torch.no_grad():  # No need to calculate gradients
    output = model_loaded(encoded_input)



# Hybrid Tokenizer 

In [32]:
from heapq import nlargest
import collections
import numpy as np
import re
import torch
from torch.utils.data import Dataset

def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def tokenize(text):
    return re.findall(r'\b\w+\b|[\s\.,!?;]', text)

def build_vocab(tokens, max_vocab_size=10000):
    token_freqs = Counter(tokens)
    sorted_tokens = sorted(token_freqs.items(), key=lambda x: (-x[1], x[0]))
    # Ensure special tokens are included
    vocab = {"[PAD]": 0, "[UNK]": 1, "[CLS]": 2, "[SEP]": 3, "[MASK]": 4}
    for token, _ in sorted_tokens[:max_vocab_size - len(vocab)]:
        vocab[token] = len(vocab)
    return vocab

# Tokenizing Code
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_seq_len= 512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer.encode(text)[:self.max_seq_len]         
        labels = encoded_text.copy()  # Copy encoded text for labels

        # Apply dynamic masking
        masked_indices = np.random.rand(len(encoded_text)) < 0.15
        for i in range(len(encoded_text)):
            if masked_indices[i]:
                encoded_text[i] = self.tokenizer.vocab["[MASK]"]

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        attention_mask = [1] * len(encoded_text) + [0] * padding_length
        encoded_text += [self.tokenizer.vocab["[PAD]"]] * padding_length
        labels += [-100] * padding_length  # Use -100 for padding positions

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }
    
class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup
        self.frequency = 0  # Track frequency of subwords for simplified ranking


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency  # Assign frequency at the leaf node


    def find_subwords(self, token):
        """
        Finds the most probable subwords for a given token based on frequency.
        """
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Convert frequency to a simple probability for demonstration
                probability = current_node.frequency / sum(node.frequency for node in collected_subwords)
                collected_subwords.append((subword, probability, current_node.token_id))

            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, collected_subwords)

        dfs(node)
        # Instead of selecting based purely on frequency, consider the 'probability'
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword[2] for subword in best_subwords][:5] if best_subwords else [self.unk_token_id]

    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

    def compute_failure_links(self):
        root = self.root
        root.failure_link = root  # The root's failure link points to itself
        queue = [root]

        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)

                # Set the failure link for the child_node
                if current_node == root:
                    child_node.failure_link = root
                else:
                    # Follow the current node's failure link to find the longest suffix for the child_node
                    failure_candidate = current_node.failure_link
                    while failure_candidate != root and char not in failure_candidate.children:
                        failure_candidate = failure_candidate.failure_link
                    child_node.failure_link = failure_candidate.children.get(char, root)




class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.
        Args:
            corpus: A text corpus represented as a list of strings.
        """
        self.vocab = self.init_vocab(corpus)
        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            if not pairs:  # Check if there are no more pairs to merge
                break
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print("Merging: {}".format(best))  # Track most frequent pair in each iteration

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies, tokenized at the character level."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.strip().split()
            for word in words:
                # No longer splitting the word into characters with spaces
                vocab[word + " </w>"] += 1  # Marking end of word with </w>
        return vocab

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None
        self.failure_pops = None
        self.vocab = {}  # Initialize vocab as an empty dictionary

    def train(self, corpus):
        self.vocab = self._build_vocabulary(corpus)
        self._precompute(self.vocab) 

    def _encode(self, text):
        """
        Tokenizes input text into subwords, using a trie to find the most probable subword sequences.
        """
        tokens = text.split()  # Simplified tokenization
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        # Adjusted to handle the corpus as a list
        if model_type == "unigram":
            # Aggregate tokens from all texts in the corpus
            all_tokens = []
            for text in corpus:
                tokens = self._unigram_tokenize(text)
                all_tokens.extend(tokens)
            vocabulary = self._build_unigram_vocab(all_tokens, vocab_size)
        elif model_type == "bpe":
            # Assuming a similar adjustment for BPE if necessary
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _unigram_tokenize(self, text):
        # Now properly handles a single text/string
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', text)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Initialize the trie
        self.trie = Trie()
        for token_id, (token, frequency) in enumerate(vocabulary.items()):
            # Insert tokens into the trie
            self.trie.insert(token, token_id, frequency)

        # Compute failure links after the trie has been fully constructed
        self.trie.compute_failure_links() 

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root


class WordPieceTokenizer:
    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def preprocess(self, text):
        # Lowercase the text to standardize it
        text = text.lower()
        
        # Replace or remove special characters as needed
        # For demonstration, replacing common quotation marks and removing non-ASCII characters
        text = text.replace("“", '"').replace("”", '"')
        text = re.sub(r"[^\x00-\x7F]+", "", text)
        
        # Whitespace normalization
        text = re.sub(r"\s+", " ", text).strip()
        
        # Optional: Handling numbers (here, we just replace them with a placeholder)
        text = re.sub(r"\d+", "<num>", text)
        
        return text


    def tokenize(self, text):
        output_tokens = []
        text = self.preprocess(text)  # Apply preprocessing
        for word in text.strip().split():
            if len(word) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            if word in self.vocab:
                output_tokens.append(word)
            else:
                is_bad, sub_tokens = False, []
                start = 0
                while start < len(word):
                    end = len(word)
                    cur_substr = None
                    while start < end:
                        substr = "##" + word[start:end] if start > 0 else word[start:end]
                        if substr in self.vocab:
                            cur_substr = substr
                            break
                        end -= 1

                    if cur_substr is None:
                        is_bad = True
                        break

                    sub_tokens.append(cur_substr)
                    start = end

                if is_bad:
                    output_tokens.append(self.unk_token)
                else:
                    output_tokens.extend(sub_tokens)

        return output_tokens




class ContextualTokenizer:
    def __init__(self, corpus, vocab_size=10000, max_seq_len=512):
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.bpe = BPE()
        self.sentence_piece = SentencePiece()
        self.word_piece = None  # Will be initialized post-BPE and SentencePiece training
        self.tokenizer_vocab = None  # Final vocabulary used by WordPieceTokenizer

    def train_tokenizers(self):
        # Train BPE and build its vocabulary
        print("Training BPE...")
        self.bpe.train(self.corpus)
        bpe_vocab = self.bpe.vocab

        # Train SentencePiece and build its vocabulary
        print("Training SentencePiece...")
        self.sentence_piece.train(self.corpus)
        sentence_piece_vocab = self.sentence_piece._build_vocabulary(self.corpus, self.vocab_size)

        # Merge BPE and SentencePiece vocabularies for WordPiece
        merged_vocab = {**bpe_vocab, **sentence_piece_vocab}
        self.tokenizer_vocab = {token: idx for idx, token in enumerate(merged_vocab.keys())}
        # After merging the vocabularies
        print("Sample from merged vocabulary:", list(self.tokenizer_vocab.keys())[:100])
        # Right after merging vocabularies
        print("Merged vocab size:", len(self.tokenizer_vocab))

        # Initialize WordPiece with the merged vocabulary
        print("Initializing WordPiece...")
        self.word_piece = WordPieceTokenizer(self.tokenizer_vocab)

    def tokenize_corpus(self):
        # Tokenize the corpus using WordPiece
        tokenized_corpus = [self.word_piece.tokenize(text) for text in self.corpus]
        return tokenized_corpus

    def create_dataset(self, texts):
        # Utilize the WordPieceTokenizer's encoded tokens to create a dataset
        dataset = TextDataset(texts, self.word_piece, self.max_seq_len)
        return dataset



# Test on my own txt file
# Load the corpus
text = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")

# Initialize your tokenizer with the corpus (adjust as needed)
contextual_tokenizer = ContextualTokenizer(corpus=[text], vocab_size=10000, max_seq_len=512)
contextual_tokenizer.train_tokenizers()

# Tokenize the corpus and create a dataset
tokenized_corpus = contextual_tokenizer.tokenize_corpus()
dataset = contextual_tokenizer.create_dataset(tokenized_corpus)
print(tokenized_corpus)


Training BPE...
Merging: ('the', '</w>')
Merging: ('of', '</w>')
Merging: ('to', '</w>')
Merging: ('and', '</w>')
Merging: ('a', '</w>')
Merging: ('is', '</w>')
Merging: ('in', '</w>')
Merging: ('that', '</w>')
Merging: ('on', '</w>')
Merging: ('for', '</w>')
Training SentencePiece...
Sample from merged vocabulary: ['How </w>', 'transferable </w>', 'are </w>', 'features </w>', 'in</w>', 'deep </w>', 'neural </w>', 'networks? </w>', 'Jason </w>', 'Yosinski,1 </w>', 'Jeff </w>', 'Clune,2 </w>', 'Yoshua </w>', 'Bengio,3 </w>', 'and</w>', 'Hod </w>', 'Lipson4 </w>', '1 </w>', 'Dept. </w>', 'Computer </w>', 'Science, </w>', 'Cornell </w>', 'University </w>', '2 </w>', 'of</w>', 'Wyoming </w>', '3 </w>', 'Science </w>', '& </w>', 'Operations </w>', 'Research, </w>', 'Montreal </w>', '4 </w>', 'Mechanical </w>', 'Aerospace </w>', 'Engineering, </w>', 'Abstract </w>', 'Many </w>', 'networks </w>', 'trained </w>', 'on</w>', 'natural </w>', 'images </w>', 'exhibit </w>', 'a</w>', 'curious </w>',

# Hybrid v2

In [ ]:
from heapq import nlargest
import collections
import numpy as np
import re
import torch
import torch.nn as nn 


def cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return 0.5 * (1. + math.cos(math.pi * progress)) 

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def combined_loss(output, target, model, l2_reg_strength=1.0, l1_reg_strength=0.0):
    task_loss = nn.CrossEntropyLoss()(output, target)  
    regularization_loss = 0

    for param in model.parameters():
        if isinstance(param, nn.Parameter):  
            regularization_loss += param.pow(2).sum() * l2_reg_strength  # L2
            regularization_loss += param.abs().sum() * l1_reg_strength  # L1

    return task_loss + regularization_loss



class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup
        self.frequency = 0  # Track frequency of subwords for simplified ranking


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency  # Assign frequency at the leaf node


    def find_subwords(self, token):
        """
        Finds the most probable subwords for a given token based on frequency.
        """
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Convert frequency to a simple probability for demonstration
                probability = current_node.frequency / sum(node.frequency for node in collected_subwords)
                collected_subwords.append((subword, probability, current_node.token_id))

            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, collected_subwords)

        dfs(node)
        # Instead of selecting based purely on frequency, consider the 'probability'
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword[2] for subword in best_subwords][:5] if best_subwords else [self.unk_token_id]

    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

    def compute_failure_links(self):
        root = self.root
        root.failure_link = root  # The root's failure link points to itself
        queue = [root]

        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)

                # Set the failure link for the child_node
                if current_node == root:
                    child_node.failure_link = root
                else:
                    # Follow the current node's failure link to find the longest suffix for the child_node
                    failure_candidate = current_node.failure_link
                    while failure_candidate != root and char not in failure_candidate.children:
                        failure_candidate = failure_candidate.failure_link
                    child_node.failure_link = failure_candidate.children.get(char, root)


class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.
        Args:
            corpus: A text corpus represented as a list of strings.
        """
        self.vocab = self.init_vocab(corpus)
        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            if not pairs:  # Check if there are no more pairs to merge
                break
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print("Merging: {}".format(best))  # Track most frequent pair in each iteration

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies, tokenized at the character level."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.strip().split()
            for word in words:
                # No longer splitting the word into characters with spaces
                vocab[word + " </w>"] += 1  # Marking end of word with </w>
        return vocab

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None
        self.failure_pops = None
        self.vocab = {}  # Initialize vocab as an empty dictionary

    def train(self, corpus):
        self.vocab = self._build_vocabulary(corpus)
        self._precompute(self.vocab) 

    def _encode(self, text):
        """
        Tokenizes input text into subwords, using a trie to find the most probable subword sequences.
        """
        tokens = text.split()  # Simplified tokenization
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        # Adjusted to handle the corpus as a list
        if model_type == "unigram":
            # Aggregate tokens from all texts in the corpus
            all_tokens = []
            for text in corpus:
                tokens = self._unigram_tokenize(text)
                all_tokens.extend(tokens)
            vocabulary = self._build_unigram_vocab(all_tokens, vocab_size)
        elif model_type == "bpe":
            # Assuming a similar adjustment for BPE if necessary
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _unigram_tokenize(self, text):
        # Now properly handles a single text/string
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', text)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Initialize the trie
        self.trie = Trie()
        for token_id, (token, frequency) in enumerate(vocabulary.items()):
            # Insert tokens into the trie
            self.trie.insert(token, token_id, frequency)

        # Compute failure links after the trie has been fully constructed
        self.trie.compute_failure_links() 

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root


class WordPieceTokenizer:
    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def preprocess(self, text):
        # Lowercase the text to standardize it
        text = text.lower()
        
        # Replace or remove special characters as needed
        # For demonstration, replacing common quotation marks and removing non-ASCII characters
        text = text.replace("“", '"').replace("”", '"')
        text = re.sub(r"[^\x00-\x7F]+", "", text)
        
        # Whitespace normalization
        text = re.sub(r"\s+", " ", text).strip()
        
        # Optional: Handling numbers (here, we just replace them with a placeholder)
        text = re.sub(r"\d+", "<num>", text)
        
        return text


    def tokenize(self, text):
        output_tokens = []
        text = self.preprocess(text)  # Apply preprocessing
        for word in text.strip().split():
            if len(word) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            if word in self.vocab:
                output_tokens.append(word)
            else:
                is_bad, sub_tokens = False, []
                start = 0
                while start < len(word):
                    end = len(word)
                    cur_substr = None
                    while start < end:
                        substr = "##" + word[start:end] if start > 0 else word[start:end]
                        if substr in self.vocab:
                            cur_substr = substr
                            break
                        end -= 1

                    if cur_substr is None:
                        is_bad = True
                        break

                    sub_tokens.append(cur_substr)
                    start = end

                if is_bad:
                    output_tokens.append(self.unk_token)
                else:
                    output_tokens.extend(sub_tokens)

        return output_tokens



class TransformerMergingNetwork(nn.Module):
    def __init__(self, combined_vocab, vocab_size_bpe, vocab_size_sp, vocab_size_wp, 
                 embedding_dim, nhead, dim_feedforward, freq_threshold, smaller_embed_dim, 
                 max_seq_len=512, initial_max_frequency=1000):
        super().__init__()

        self.token_embedding_bpe = nn.Embedding(vocab_size_bpe, embedding_dim)
        self.token_embedding_sp = nn.Embedding(vocab_size_sp, embedding_dim)
        self.token_embedding_wp = nn.Embedding(vocab_size_wp, embedding_dim)
        self.max_frequency = initial_max_frequency
        self.freq_embedding = nn.Embedding(self.max_frequency, embedding_dim)  

        self.positional_encoding = nn.Parameter(torch.randn(3 * max_seq_len, embedding_dim)) 
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead), 
            num_layers=num_layers
        )

        self.output_layer = nn.Linear(embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, bpe_tokens, bpe_freqs, sp_tokens, sp_freqs, wp_tokens, wp_freqs):
        embedded_bpe = self.token_embedding_bpe(bpe_tokens) + self.positional_encoding[:len(bpe_tokens)] 
        embedded_sp = self.token_embedding_sp(sp_tokens) + self.positional_encoding[len(bpe_tokens):(len(bpe_tokens) + len(sp_tokens))]
        embedded_wp = self.token_embedding_wp(wp_tokens) + self.positional_encoding[(len(bpe_tokens) + len(sp_tokens)):] 
        if torch.any(torch.cat((bpe_freqs, sp_freqs, wp_freqs)) >= self.max_frequency):   # Combined check
            self._increase_max_frequency() 
        embedded_freqs = self.freq_embedding(torch.cat((bpe_freqs, sp_freqs, wp_freqs)))

        # Concatenate all embeddings across tokenizers in sequence with frequencies
        combined_input = torch.cat((embedded_bpe, embedded_sp, embedded_wp, embedded_freqs), dim=1)

        encoded_output = self.transformer_encoder(combined_input)
        merge_weights = self.sigmoid(self.output_layer(encoded_output))
        return merge_weights

    def _increase_max_frequency(self):
        new_max_frequency = int(self.max_frequency * 1.5)  
        self.freq_embedding = nn.Embedding(new_max_frequency, self.freq_embedding.embedding_dim)  # Resize 
        self.max_frequency = new_max_frequency 

    def merging_loss(self, bpe_embeddings, sp_embeddings, wp_embeddings, merge_weights, margin=1.0):
        """
        Calculates the merging loss.

        Args:
            bpe_embeddings:  Embeddings for BPE tokens from the combined output of Transformers network.
            sp_embeddings: Embeddings for SentencePiece tokens.
            wp_embeddings: Embeddings for WordPiece tokens.
            merge_weights: Predicted merge weights (output of the Transformer model).
            margin: Margin for contrastive loss component.

        Returns:
            loss: The calculated merging loss.
        """

        # Cross-Tokenizer Presence Loss
        presence_loss = torch.mean(merge_weights**2)  # Squared weights favor high confidence merges

        # Semantic Similarity Loss (using Contrastive Loss as an example)
        similarity_loss = 0.0

        for bpe_emb, sp_emb, wp_emb in zip(bpe_embeddings, sp_embeddings, wp_embeddings):
            # Calculate pairwise distances between embeddings (cosine distance example)
            dist_bpe_sp = 1 - torch.cosine_similarity(bpe_emb, sp_emb) 
            dist_bpe_wp = 1 - torch.cosine_similarity(bpe_emb, wp_emb)  
            dist_sp_wp = 1 - torch.cosine_similarity(sp_emb, wp_emb) 

            # Contrastive terms: Encourage matching pairs, discourage non-matching
            similarity_loss += torch.maximum(0.0, dist_bpe_sp - margin)  
            similarity_loss += torch.maximum(0.0, dist_bpe_wp - margin)
            similarity_loss += torch.maximum(0.0, dist_sp_wp - margin)

        # Combine with appropriate weights
        alpha = 0.6  # Example - Adjust as needed
        total_loss = alpha * presence_loss + (1 - alpha) * similarity_loss  

        return total_loss

In [49]:
from heapq import nlargest
import collections
import numpy as np
import re
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader


def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = [line.strip() for line in file.readlines()]
    return texts


def tokenize(text):
    return re.findall(r'\b\w+\b|[\s\.,!?;]', text)

def build_vocab(tokens, max_vocab_size=10000):
    token_freqs = Counter(tokens)
    sorted_tokens = sorted(token_freqs.items(), key=lambda x: (-x[1], x[0]))
    # Ensure special tokens are included
    vocab = {"[PAD]": 0, "[UNK]": 1, "[CLS]": 2, "[SEP]": 3, "[MASK]": 4}
    for token, _ in sorted_tokens[:max_vocab_size - len(vocab)]:
        vocab[token] = len(vocab)
    return vocab


# Tokenizing Code
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_seq_len= 512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded_text = self.tokenized_texts[idx][:self.max_seq_len]
        labels = encoded_text.copy()  # Copy encoded text for labels

        # Apply dynamic masking
        masked_indices = np.random.rand(len(encoded_text)) < 0.15
        for i in range(len(encoded_text)):
            if masked_indices[i]:
                encoded_text[i] = self.tokenizer.vocab["[MASK]"]

        # Padding
        padding_length = self.max_seq_len - len(encoded_text)
        attention_mask = [1] * len(encoded_text) + [0] * padding_length
        encoded_text += [self.tokenizer.vocab["[PAD]"]] * padding_length
        labels += [-100] * padding_length  # Use -100 for padding positions

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }
    
class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup
        self.frequency = 0  # Track frequency of subwords for simplified ranking


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency  # Assign frequency at the leaf node


    def find_subwords(self, token):
        """
        Finds the most probable subwords for a given token based on frequency.
        """
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Convert frequency to a simple probability for demonstration
                probability = current_node.frequency / sum(node.frequency for node in collected_subwords)
                collected_subwords.append((subword, probability, current_node.token_id))

            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, collected_subwords)

        dfs(node)
        # Instead of selecting based purely on frequency, consider the 'probability'
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword[2] for subword in best_subwords][:5] if best_subwords else [self.unk_token_id]

    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

    def compute_failure_links(self):
        root = self.root
        root.failure_link = root  # The root's failure link points to itself
        queue = [root]

        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)

                # Set the failure link for the child_node
                if current_node == root:
                    child_node.failure_link = root
                else:
                    # Follow the current node's failure link to find the longest suffix for the child_node
                    failure_candidate = current_node.failure_link
                    while failure_candidate != root and char not in failure_candidate.children:
                        failure_candidate = failure_candidate.failure_link
                    child_node.failure_link = failure_candidate.children.get(char, root)


class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.
        Args:
            corpus: A text corpus represented as a list of strings.
        """
        self.vocab = self.init_vocab(corpus)
        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            if not pairs:  # Check if there are no more pairs to merge
                break
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print("Merging: {}".format(best))  # Track most frequent pair in each iteration

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies, tokenized at the character level."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.strip().split()
            for word in words:
                # No longer splitting the word into characters with spaces
                vocab[word + " </w>"] += 1  # Marking end of word with </w>
        return vocab

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab

class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None
        self.failure_pops = None
        self.vocab = {}  # Initialize vocab as an empty dictionary

    def train(self, corpus):
        self.vocab = self._build_vocabulary(corpus)
        self._precompute(self.vocab) 

    def _encode(self, text):
        """
        Tokenizes input text into subwords, using a trie to find the most probable subword sequences.
        """
        tokens = text.split()  # Simplified tokenization
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        # Adjusted to handle the corpus as a list
        if model_type == "unigram":
            # Aggregate tokens from all texts in the corpus
            all_tokens = []
            for text in corpus:
                tokens = self._unigram_tokenize(text)
                all_tokens.extend(tokens)
            vocabulary = self._build_unigram_vocab(all_tokens, vocab_size)
        elif model_type == "bpe":
            # Assuming a similar adjustment for BPE if necessary
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _unigram_tokenize(self, text):
        # Now properly handles a single text/string
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', text)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

    def _build_unigram_vocab(self, tokens, vocab_size):
        # Count token frequencies
        token_freqs = collections.Counter(tokens)
        # Select the most frequent tokens up to vocab_size
        vocab = {token: idx for idx, (token, _) in enumerate(token_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Initialize the trie
        self.trie = Trie()
        for token_id, (token, frequency) in enumerate(vocabulary.items()):
            # Insert tokens into the trie
            self.trie.insert(token, token_id, frequency)

        # Compute failure links after the trie has been fully constructed
        self.trie.compute_failure_links() 

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root


class WordPieceTokenizer:
    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def preprocess(self, text):
        # Lowercase the text to standardize it
        text = text.lower()
        
        
        # Replace or remove special characters as needed
        # For demonstration, replacing common quotation marks and removing non-ASCII characters
        text = text.replace("“", '"').replace("”", '"')
        text = re.sub(r"[^\x00-\x7F]+", "", text)
        
        # Whitespace normalization
        text = re.sub(r"\s+", " ", text).strip()
        
        # Optional: Handling numbers (here, we just replace them with a placeholder)
        text = re.sub(r"\d+", "<num>", text)
        
        return text


    def tokenize(self, text):
        output_tokens = []
        text = self.preprocess(text)  # Apply preprocessing
        for word in text.strip().split():
            if len(word) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            if word in self.vocab:
                output_tokens.append(word)
            else:
                is_bad, sub_tokens = False, []
                start = 0
                while start < len(word):
                    end = len(word)
                    cur_substr = None
                    while start < end:
                        substr = "##" + word[start:end] if start > 0 else word[start:end]
                        if substr in self.vocab:
                            cur_substr = substr
                            break
                        end -= 1

                    if cur_substr is None:
                        is_bad = True
                        break

                    sub_tokens.append(cur_substr)
                    start = end

                if is_bad:
                    output_tokens.append(self.unk_token)
                else:
                    output_tokens.extend(sub_tokens)

        return output_tokens



class ContextualTokenizer:
    def __init__(self, corpus, vocab_size=10000, max_seq_len=512):
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.bpe = BPE()
        self.sentence_piece = SentencePiece()
        self.word_piece = None  # Will be initialized post-BPE and SentencePiece training
        self.tokenizer_vocab = None  # Final vocabulary used by WordPieceTokenizer

    def train_tokenizers(self):
        # Train BPE and build its vocabulary
        print("Training BPE...")
        self.bpe.train(self.corpus)
        bpe_vocab = self.bpe.vocab

        # Train SentencePiece and build its vocabulary
        print("Training SentencePiece...")
        self.sentence_piece.train(self.corpus)
        sentence_piece_vocab = self.sentence_piece._build_vocabulary(self.corpus, self.vocab_size)

        # Merge BPE and SentencePiece vocabularies for WordPiece
        merged_vocab = {**bpe_vocab, **sentence_piece_vocab}
        self.tokenizer_vocab = {token: idx for idx, token in enumerate(merged_vocab.keys())}
        # After merging the vocabularies
        print("Sample from merged vocabulary:", list(self.tokenizer_vocab.keys())[:100])
        # Right after merging vocabularies
        print("Merged vocab size:", len(self.tokenizer_vocab))

        # Initialize WordPiece with the merged vocabulary
        print("Initializing WordPiece...")
        self.word_piece = WordPieceTokenizer(self.tokenizer_vocab)

    def tokenize_corpus(self):
        # Tokenize the corpus using WordPiece
        tokenized_corpus = [self.word_piece.tokenize(text) for text in self.corpus]
        return tokenized_corpus

    def create_dataset(self, texts):
        # Utilize the WordPieceTokenizer's encoded tokens to create a dataset
        dataset = TextDataset(texts, self.word_piece, self.max_seq_len)
        return dataset

# Encoding Transformer Code

class AdaptiveDropoutLayer(nn.Module):
    def __init__(self, init_dropout_rate=0.1):
        super(AdaptiveDropoutLayer, self).__init__()
        # Use logit transformation for stability
        self.log_alpha = nn.Parameter(torch.tensor(math.log(init_dropout_rate / (1 - init_dropout_rate))).float())

    def forward(self, x):
        p = torch.sigmoid(self.log_alpha)
        # Convert p from a tensor to a float
        p_value = p.item()  # This extracts the scalar value as a Python float
        return nn.functional.dropout(x, p=p_value, training=self.training)


class AdaptiveWeightDecayOptimizer(optim.Optimizer):
    def __init__(self, params, lr=1e-3, init_l2_strength=0.01):
        super().__init__(params, {'lr': lr})
        self.log_l2_strength = nn.Parameter(torch.tensor(math.log(init_l2_strength)).float())

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            weight_decay = torch.exp(self.log_l2_strength)  
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay) 
                p.update(d_p, group['lr']) 

        return loss

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len

        # Create positional encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add a batch dimension (B x T x C)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: Tensor of shape [Batch Size, Sequence Length, Embedding Dimension]
        # Adjust positional encoding to match the input size and device
        pe = self.pe[:, :x.size(1)]
        # Assuming x is on the correct device, pe will be automatically aligned to the same device
        return pe


class MultiHeadLinformerAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, k=None):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.k = k if k is not None else embed_dim // num_heads  # Projection dimension per head

        self.key_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.value_projections = nn.Linear(embed_dim, self.k * num_heads)
        self.out_projection = nn.Linear(self.k * num_heads, embed_dim)

    def forward(self, query):
        batch_size, seq_len, _ = query.size()
        
        # Project keys and values
        keys = self.key_projections(query)
        values = self.value_projections(query)
        
        # Reshape into [batch_size, num_heads, seq_len, k]
        keys = keys.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        values = values.reshape(batch_size, seq_len, self.num_heads, self.k).transpose(1, 2)
        
        # Calculate attention (scaled dot-product attention)
        # Scaling by the square root of the depth of the key vectors to prevent large values in the dot product
        # which could push the softmax function into regions where it has extremely small gradients
        keys = keys / (self.k ** 0.5)
        attention_scores = torch.softmax(torch.matmul(keys, values.transpose(-2, -1)), dim=-1)
        
        # Apply attention to values
        out = torch.matmul(attention_scores, values)
        
        # Concatenate heads and project back to original embedding dimension
        out = out.transpose(1, 2).reshape(batch_size, seq_len, self.num_heads * self.k)
        out = self.out_projection(out)
        
        return out


class AdaptiveEmbeddingLayer(nn.Module):
    def __init__(self, vocab,  vocab_size, freq_threshold, large_embed_dim, small_embed_dim, max_seq_len):
        super(AdaptiveEmbeddingLayer, self).__init__()
        self.vocab = vocab
        self.vocab_size = vocab_size
        self.freq_threshold = freq_threshold
        self.large_embed_dim = large_embed_dim
        self.small_embed_dim = small_embed_dim
        self.max_seq_len = max_seq_len

        self.split_vocab(vocab, freq_threshold)  

        self.frequent_embeddings = nn.Embedding(num_embeddings=len(self.frequent_vocab), embedding_dim=large_embed_dim)
        self.infrequent_embeddings = nn.Embedding(num_embeddings=len(self.infrequent_vocab), embedding_dim=small_embed_dim)
        self.infrequent_projection = nn.Linear(small_embed_dim, large_embed_dim)
        self.positional_embeddings = PositionalEncoding(large_embed_dim, max_seq_len)


    def split_vocab(self, vocab, freq_threshold):
        token_counts = [(token, count) for token, count in vocab.items()]
        token_counts.sort(key=lambda x: -x[1])  # Sort by frequency
        split_point = next(i for i, (_, count) in enumerate(token_counts) if count < freq_threshold)
        
        self.frequent_vocab = dict(token_counts[:split_point])
        self.infrequent_vocab = dict(token_counts[split_point:])

    def forward(self, token_ids):
        device = token_ids.device
        seq_len = token_ids.size(1)
        batch_size = token_ids.size(0)  # Obtain batch size from token_ids tensor

        # Initialize embeddings tensor
        embeddings = torch.zeros(token_ids.shape[0], seq_len, self.large_embed_dim, device=device)

        # Map token_ids to indices for frequent and infrequent vocab
        frequent_indices = torch.zeros_like(token_ids)
        infrequent_indices = torch.zeros_like(token_ids)
        
        for token_id, index in self.vocab.items():
            mask = token_ids == token_id
            if token_id in self.frequent_vocab:
                # Map to index in frequent_vocab
                frequent_indices[mask] = self.frequent_vocab[token_id]
            elif token_id in self.infrequent_vocab:
                # Map to index in infrequent_vocab
                infrequent_indices[mask] = self.infrequent_vocab[token_id]

        # Create masks for frequent and infrequent tokens
        frequent_mask = frequent_indices > 0
        infrequent_mask = infrequent_indices > 0

        # Embed frequent tokens
        if frequent_mask.any():
            frequent_embeddings = self.frequent_embeddings(frequent_indices[frequent_mask])
            embeddings[frequent_mask] = frequent_embeddings

        # Embed and project infrequent tokens
        if infrequent_mask.any():
            infrequent_embeddings = self.infrequent_embeddings(infrequent_indices[infrequent_mask])
            infrequent_embeddings_projected = self.infrequent_projection(infrequent_embeddings)
            embeddings[infrequent_mask] = infrequent_embeddings_projected

        # Apply positional embeddings
        position_ids = torch.arange(0, seq_len, dtype=torch.long, device=device).unsqueeze(0)
        position_embeddings = self.positional_embeddings(position_ids)  # Generate for seq_len

        # Ensure positional embeddings are broadcastable to the embeddings tensor
        # This step may not be necessary if your positional embeddings are already correctly shaped
        if position_embeddings.size(0) != batch_size:
            position_embeddings = position_embeddings.expand(batch_size, -1, -1)

        print(f"Embeddings shape: {embeddings.shape}")
        print(f"Positional embeddings shape: {position_embeddings.shape}")
        embeddings += position_embeddings

        return embeddings


class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super(TransformerEncoderLayer, self).__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadLinformerAttention(embed_dim=d_model, num_heads=nhead)
        self.dropout1 = AdaptiveDropoutLayer()  # Use AdaptiveDropoutLayer
        self.norm2 = nn.LayerNorm(d_model)

        self.ffnn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            AdaptiveDropoutLayer(),  # Use AdaptiveDropoutLayer here as well
            nn.Linear(dim_feedforward, d_model),
        )
        self.dropout2 = AdaptiveDropoutLayer()  # And here
 
    def forward(self, src, src_mask=None):
        src2 = self.norm1(src)
        attn_output = self.attn(src2)
        src = src + self.dropout1(attn_output)
        src2 = self.norm2(src)
        src = src + self.dropout2(self.ffnn(src2))
        return src


class Pooler(nn.Module):
    def __init__(self, d_model):
        super(Pooler, self).__init__()
        self.linear = nn.Linear(d_model, d_model)
        self.activation = nn.Tanh()

    def forward(self, input_tensor):
        # Assuming input_tensor is of shape [batch_size, seq_len, d_model], take the first token's representations
        first_token_tensor = input_tensor[:, 0]
        pooled_output = self.linear(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output


class TransformerMergingNetwork(nn.Module):
    def __init__(self, combined_vocab, vocab_size_bpe, vocab_size_sp, vocab_size_wp, 
                 embedding_dim, nhead, dim_feedforward, freq_threshold, smaller_embed_dim, 
                 max_seq_len=512, initial_max_frequency=1000):
        super().__init__()


        # Adaptive Embedding Layers
        self.token_embedding_bpe = AdaptiveEmbeddingLayer(
            vocab=vocab_size_bpe, 
            vocab_size=len(vocab_size_bpe),  # Size based on the combined vocabulary
            freq_threshold=freq_threshold,
            large_embed_dim=embedding_dim,
            small_embed_dim=smaller_embed_dim,
            max_seq_len=max_seq_len
        )
        self.token_embedding_sp = AdaptiveEmbeddingLayer(
            vocab=vocab_size_sp, 
            vocab_size=len(vocab_size_sp),  # Size based on the combined vocabulary
            freq_threshold=freq_threshold,
            large_embed_dim=embedding_dim,
            small_embed_dim=smaller_embed_dim,
            max_seq_len=max_seq_len
        )
        self.token_embedding_wp = AdaptiveEmbeddingLayer(
            vocab=vocab_size_wp, 
            vocab_size=len(vocab_size_wp),  # Size based on the combined vocabulary
            freq_threshold=freq_threshold,
            large_embed_dim=embedding_dim,
            small_embed_dim=smaller_embed_dim,
            max_seq_len=max_seq_len
        )

        # Positional Encoding (Adapt if your implementation differs)
        self.positional_encoding = nn.Parameter(torch.randn(3 * max_seq_len, embedding_dim)) 

        # Transformer Encoder with Linformer Attention
        self.transformer_encoder = nn.TransformerEncoder(
            TransformerEncoderLayer(
                d_model=embedding_dim, 
                nhead=nhead, 
                dim_feedforward=dim_feedforward,
                dropout=0.1  # Example dropout - may need adjustment
            ),
            num_layers=1  # Replace with the desired number of layers
        )

        # Output and Loss Calculation Components
        self.output_layer = nn.Linear(embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.max_frequency = initial_max_frequency  

        # Frequency embedding - keep to embed frequencies
        self.freq_embedding = nn.Embedding(self.max_frequency, embedding_dim) 

    def forward(self, bpe_tokens, bpe_freqs, sp_tokens, sp_freqs, wp_tokens, wp_freqs):
        # Compute embeddings for each tokenizer's tokens
        embedded_bpe = self.token_embedding_bpe(bpe_tokens) + self.positional_encoding[:len(bpe_tokens)]
        embedded_sp = self.token_embedding_sp(sp_tokens) + self.positional_encoding[len(bpe_tokens):(len(bpe_tokens) + len(sp_tokens))]
        embedded_wp = self.token_embedding_wp(wp_tokens) + self.positional_encoding[(len(bpe_tokens) + len(sp_tokens)):]
        
        # Embedding frequencies
        embedded_freqs = self.freq_embedding(torch.cat((bpe_freqs, sp_freqs, wp_freqs)))
        
        # Combine embeddings for input to the transformer encoder
        combined_input = torch.cat((embedded_bpe, embedded_sp, embedded_wp, embedded_freqs), dim=0)  # Adjust dim as needed
        
        # Pass combined embeddings through the transformer encoder
        encoded_output = self.transformer_encoder(combined_input.unsqueeze(0))  # Adjust shape as needed
        
        # Compute merge weights from encoded output
        merge_weights = self.sigmoid(self.output_layer(encoded_output.squeeze(0)))  # Adjust squeezing as needed
        
        # Return merge weights and the original token embeddings
        return merge_weights, embedded_bpe, embedded_sp, embedded_wp


    def merging_loss(self, bpe_embeddings, sp_embeddings, wp_embeddings, merge_weights, margin=1.0):
        """
        Calculates the merging loss.

        Args:
            bpe_embeddings:  Embeddings for BPE tokens from the combined output of Transformers network.
            sp_embeddings: Embeddings for SentencePiece tokens.
            wp_embeddings: Embeddings for WordPiece tokens.
            merge_weights: Predicted merge weights (output of the Transformer model).
            margin: Margin for contrastive loss component.

        Returns:
            loss: The calculated merging loss.
        """

        # Cross-Tokenizer Presence Loss
        presence_loss = torch.mean(merge_weights**2)  # Squared weights favor high confidence merges

        # Semantic Similarity Loss (using Contrastive Loss as an example)
        similarity_loss = 0.0

        for bpe_emb, sp_emb, wp_emb in zip(bpe_embeddings, sp_embeddings, wp_embeddings):
            # Calculate pairwise distances between embeddings (cosine distance example)
            dist_bpe_sp = 1 - torch.cosine_similarity(bpe_emb, sp_emb) 
            dist_bpe_wp = 1 - torch.cosine_similarity(bpe_emb, wp_emb)  
            dist_sp_wp = 1 - torch.cosine_similarity(sp_emb, wp_emb) 

            # Contrastive terms: Encourage matching pairs, discourage non-matching
            similarity_loss += torch.maximum(0.0, dist_bpe_sp - margin)  
            similarity_loss += torch.maximum(0.0, dist_bpe_wp - margin)
            similarity_loss += torch.maximum(0.0, dist_sp_wp - margin)

        # Combine with appropriate weights
        alpha = 0.6  # Example - Adjust as needed
        total_loss = alpha * presence_loss + (1 - alpha) * similarity_loss  

        return total_loss

def cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return 0.5 * (1. + math.cos(math.pi * progress)) 

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def combined_loss(output, target, model, merge_loss, l2_reg_strength=1.0, l1_reg_strength=0.0):
    # Task-specific loss
    task_loss = nn.CrossEntropyLoss()(output, target)
    
    # Regularization loss
    regularization_loss = 0
    for param in model.parameters():
        if param.requires_grad:
            regularization_loss += param.pow(2).sum() * l2_reg_strength  # L2 regularization
            regularization_loss += param.abs().sum() * l1_reg_strength  # L1 regularization

    # Total loss includes task loss, merge loss, and regularization loss
    total_loss = task_loss + merge_loss + regularization_loss
    return total_loss


# Assuming load_corpus now returns a list of texts
texts = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")

# Create an instance of ContextualTokenizer and train tokenizers
mixed_tokenizer = ContextualTokenizer(corpus=texts, vocab_size=10000, max_seq_len=512)
mixed_tokenizer.train_tokenizers()

# Tokenize the corpus and create a dataset
tokenized_corpus = mixed_tokenizer.tokenize_corpus()
print(f"Contextual Tokenizer tokenize_corpus: {tokenized_corpus}")
tokenized_dataset = mixed_tokenizer.create_dataset(tokenized_corpus)
dataloader = DataLoader(tokenized_dataset, batch_size= 32)
log_interval = 2
num_epochs = 5
initial_lr = 1e-4
num_warmup_steps = 1000
# Assuming dataloader is your DataLoader object and you're planning for 5 epochs
num_training_steps = len(dataloader) * num_epochs

optimizer = AdaptiveWeightDecayOptimizer(model.parameters(), lr=initial_lr)

scheduler = cosine_annealing_scheduler(optimizer, initial_lr, num_warmup_steps, num_training_steps)

# Example training loop adjustment
for epoch in range(num_epochs):
    model.train()
    total_loss_epoch = 0.0
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass: Assume model returns logits, merge weights, and embeddings
        logits, merge_weights, bpe_embeddings, sp_embeddings, wp_embeddings = model(inputs)
        
        # Compute merge loss
        merge_loss_value = model.merging_loss(bpe_embeddings, sp_embeddings, wp_embeddings, merge_weights)
        
        # Compute combined loss
        loss = combined_loss(logits, labels, model, merge_loss_value, l2_reg_strength=0.01, l1_reg_strength=0.005)
        
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update the learning rate

        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {total_loss_epoch / (batch_idx + 1):.4f}")










Training BPE...
Merging: ('the', '</w>')
Merging: ('of', '</w>')
Merging: ('to', '</w>')
Merging: ('and', '</w>')
Merging: ('a', '</w>')
Merging: ('is', '</w>')
Merging: ('in', '</w>')
Merging: ('that', '</w>')
Merging: ('on', '</w>')
Merging: ('for', '</w>')
Training SentencePiece...
Sample from merged vocabulary: ['How </w>', 'transferable </w>', 'are </w>', 'features </w>', 'in</w>', 'deep </w>', 'neural </w>', 'networks? </w>', 'Jason </w>', 'Yosinski,1 </w>', 'Jeff </w>', 'Clune,2 </w>', 'Yoshua </w>', 'Bengio,3 </w>', 'and</w>', 'Hod </w>', 'Lipson4 </w>', '1 </w>', 'Dept. </w>', 'Computer </w>', 'Science, </w>', 'Cornell </w>', 'University </w>', '2 </w>', 'of</w>', 'Wyoming </w>', '3 </w>', 'Science </w>', '& </w>', 'Operations </w>', 'Research, </w>', 'Montreal </w>', '4 </w>', 'Mechanical </w>', 'Aerospace </w>', 'Engineering, </w>', 'Abstract </w>', 'Many </w>', 'networks </w>', 'trained </w>', 'on</w>', 'natural </w>', 'images </w>', 'exhibit </w>', 'a</w>', 'curious </w>',

AttributeError: 'TextDataset' object has no attribute 'tokenized_texts'

# SmartTokenizer

In [58]:
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = [line.strip() for line in file.readlines()]
    return texts

texts = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")


class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None  # Store token IDs for efficient lookup
        self.frequency = 0  # Track frequency of subwords for simplified ranking


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency  # Assign frequency at the leaf node


    def find_subwords(self, token):
        """
        Finds the most probable subwords for a given token based on frequency.
        """
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Prevent zero division
                total_frequency = sum(node.frequency for node in collected_subwords)
                if total_frequency > 0:  
                    probability = current_node.frequency / total_frequency
                else:
                    probability = 0  # Default probability or handle differently

                collected_subwords.append(current_node) 
            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, collected_subwords)

        dfs(node)
        # Instead of selecting based purely on frequency, consider the 'probability'
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword[2] for subword in best_subwords][:5] if best_subwords else [self.unk_token_id]

    
    def _precompute(self, vocabulary):
        # Step 1: Trie Construction (remains the same)
        self.trie = Trie()  
        for token in vocabulary:
            self.trie.insert(token, self.trie.token_id)  # Assuming insertion includes token_id

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  
        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)  

                # Find Failure Link:
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None: 
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break 
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:  
                    child_node.failure_link = self.trie.root 

        # Step 3: Failure Pop Calculation 
        for node in queue:  # Could traverse in different orders; this is one option
            if node.failure_link is not None and node.token_id is None: 
                # Condition: Node does not represent a valid vocabulary item itself
                for i in range(current_node.failure_pop):
                    current_node = current_node.failure_link 
                current_node.failure_pop += node.failure_pop 

    def compute_failure_links(self):
        root = self.root
        root.failure_link = root  # The root's failure link points to itself
        queue = [root]

        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)

                # Set the failure link for the child_node
                if current_node == root:
                    child_node.failure_link = root
                else:
                    # Follow the current node's failure link to find the longest suffix for the child_node
                    failure_candidate = current_node.failure_link
                    while failure_candidate != root and char not in failure_candidate.children:
                        failure_candidate = failure_candidate.failure_link
                    child_node.failure_link = failure_candidate.children.get(char, root)


class SentencePiece:
    def __init__(self):
        self.trie = None  # Trie structure
        self.failure_links = None
        self.failure_pops = None
        self.vocab = {}  # Initialize vocab as an empty dictionary

    def train(self, corpus):
        self.vocab = self._build_vocabulary(corpus)
        self._precompute(self.vocab) 
        return self.vocab

    def _encode(self, text):
        """
        Tokenizes input text into subwords, using a trie to find the most probable subword sequences.
        """
        tokens = text.split()  # Simplified tokenization
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def _build_vocabulary(self, corpus, vocab_size=10000, model_type="unigram"):
        # Adjusted to handle the corpus as a list
        if model_type == "unigram":
            # Aggregate tokens from all texts in the corpus
            all_tokens = []
            for text in corpus:
                tokens = self._unigram_tokenize(text)
                all_tokens.extend(tokens)
            vocabulary = self._build_unigram_vocab(all_tokens, vocab_size)
        elif model_type == "bpe":
            # Assuming a similar adjustment for BPE if necessary
            vocabulary = self._build_bpe_vocab(corpus, vocab_size)
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _unigram_tokenize(self, text):
        # Now properly handles a single text/string
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', text)
        
        # Adjust tokens based on the vocabulary
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                # Token is in the vocabulary, keep it
                adjusted_tokens.append(token)
            else:
                # Token not found in the vocabulary, treat as unknown
                adjusted_tokens.append("[UNK]")
        
        return adjusted_tokens

    def _build_unigram_vocab(self, tokens, vocab_size):
        char_freqs = collections.Counter()
        for token in tokens:
            for char in token:  
                char_freqs[char] += 1

        vocab = {char: idx for idx, (char, _) in enumerate(char_freqs.most_common(vocab_size))}
        return vocab


    def _precompute(self, vocabulary):
        # Initialize the trie
        self.trie = Trie()
        for token_id, (token, frequency) in enumerate(vocabulary.items()):
            # Insert tokens into the trie
            self.trie.insert(token, token_id, frequency)

        # Compute failure links after the trie has been fully constructed
        self.trie.compute_failure_links() 

        # Step 2: Failure Link Calculation
        queue = [self.trie.root]  # Start with the root node
        while queue:
            current_node = queue.pop(0)

            # Iterate over all possible immediate children 
            for char, child_node in current_node.children.items():
                queue.append(child_node)  # Explore branches 

                # Find failure link (similar logic to Aho-Corasick)
                failure_link_candidate = current_node.failure_link 
                while failure_link_candidate is not None:  
                    if char in failure_link_candidate.children:
                        child_node.failure_link = failure_link_candidate.children[char] 
                        break
                    failure_link_candidate = failure_link_candidate.failure_link 
                else:
                    child_node.failure_link = self.trie.root  # Fallback to root

class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.
        Args:
            corpus: A text corpus represented as a list of strings.
        """
        self.vocab = self.init_vocab(corpus)
        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            if not pairs:  # Check if there are no more pairs to merge
                break
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print("Merging: {}".format(best))  # Track most frequent pair in each iteration

        return self.vocab

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies, tokenized at the character level."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.strip().split()
            for word in words:
                # No longer splitting the word into characters with spaces
                vocab[word + " </w>"] += 1  # Marking end of word with </w>
        return vocab

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab


def merge_vocabularies(bpe_vocab, sp_vocab, strategy="intersection", resolve_conflict=None):
    """
    Merges BPE and SentencePiece vocabularies.

    Args:
        bpe_vocab: BPE vocabulary (dict: subword -> index)
        sp_vocab: SentencePiece vocabulary (dict: subword -> index)
        strategy: "intersection" or "union"
        resolve_conflict: Function to determine priority when the same subword exists in both.

    Returns:
        dict: The merged vocabulary 
    """

    if strategy == "intersection":
        merged_vocab = set(bpe_vocab.keys()) & set(sp_vocab.keys())
    elif strategy == "union":
        merged_vocab = set(bpe_vocab.keys()) | set(sp_vocab.keys())
    else:
        raise ValueError("Invalid strategy. Choose 'intersection' or 'union'")

    # Handle potential conflicts with a resolution function 
    if resolve_conflict:
        final_vocab = {}
        for subword in merged_vocab:
            if subword in bpe_vocab and subword in sp_vocab:
                final_vocab[subword] = resolve_conflict(subword, bpe_vocab[subword], sp_vocab[subword])
            else:
                final_vocab[subword] = bpe_vocab.get(subword, sp_vocab[subword])
        return final_vocab

    return {subword: idx for idx, subword in enumerate(merged_vocab)}

# Example for BPE-first conflict resolution   
def prioritize_bpe(subword, bpe_index, sp_index):
    return bpe_index 


bpe = BPE()  
bpe_vocab = bpe.train(texts)  # Now actually performs BPE training

sp = SentencePiece()
sp_vocab = sp.train(texts)   # SentencePiece training is executed

print("BPE Vocab (top 20): ", list(bpe_vocab.items())[:20])
print("SP Vocab (top 20): ", list(sp_vocab.items())[:20])

print("BPE Encoding: ", bpe.encode(texts))
print("SP Encoding: ", sp._encode(texts))


merged_vocab = merge_vocabularies(bpe_vocab, sp_vocab, 
                                  strategy="intersection", # Or "union" 
                                  resolve_conflict=prioritize_bpe) 



print(merged_vocab)


class WordPieceTokenizer:
    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        output_tokens = []

        for word in text.strip().split():
            if len(word) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            if word in self.vocab:
                output_tokens.append(word)
            else:
                is_bad = False  # Flag if no suitable subwords are found
                start = 0
                sub_tokens = []

                while start < len(word):  
                    end = len(word) 
                    cur_substr = None
                    while start < end: 
                        substr = "##" + word[start:end]  # "##" prefix convention
                        if substr in self.vocab:
                            cur_substr = substr
                            break
                        end -= 1 
                    
                    if cur_substr is None: 
                        is_bad = True
                        break
                                        
                    sub_tokens.append(cur_substr)
                    start = end 

                if is_bad:
                    output_tokens.append(self.unk_token)
                else:
                    output_tokens.extend(sub_tokens) 

        return output_tokens 


Merging: ('the', '</w>')
Merging: ('of', '</w>')
Merging: ('to', '</w>')
Merging: ('and', '</w>')
Merging: ('a', '</w>')
Merging: ('is', '</w>')
Merging: ('in', '</w>')
Merging: ('that', '</w>')
Merging: ('on', '</w>')
Merging: ('for', '</w>')
BPE Vocab (top 20):  [('How </w>', 1), ('transferable </w>', 1), ('are </w>', 73), ('features </w>', 59), ('in</w>', 113), ('deep </w>', 6), ('neural </w>', 21), ('networks? </w>', 1), ('Jason </w>', 1), ('Yosinski,1 </w>', 1), ('Jeff </w>', 1), ('Clune,2 </w>', 1), ('Yoshua </w>', 1), ('Bengio,3 </w>', 1), ('and</w>', 211), ('Hod </w>', 1), ('Lipson4 </w>', 1), ('1 </w>', 17), ('Dept. </w>', 4), ('Computer </w>', 3)]
SP Vocab (top 20):  [('[', 0), ('U', 1), ('N', 2), ('K', 3), (']', 4)]


KeyboardInterrupt: 

In [71]:

class TrieNode:
    def __init__(self):
        self.children = {}
        self.token_id = None
        self.frequency = 0

class Trie:
    def __init__(self, unk_token_id=0):
        self.root = TrieNode()
        self.unk_token_id = unk_token_id

    def insert(self, token, token_id, frequency):
        node = self.root
        for char in token:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.token_id = token_id
        node.frequency = frequency

    def find_subwords(self, token):
        """Finds the most probable subwords based on frequency."""
        node = self.root
        best_subwords = []

        def dfs(current_node, subword='', collected_subwords=[]):
            if current_node.token_id is not None:
                # Update to correctly calculate total_frequency based on the structure of collected_subwords
                total_frequency = sum(n.frequency for _, _, n in collected_subwords) + current_node.frequency
                probability = current_node.frequency / total_frequency if total_frequency else 0
                collected_subwords.append((subword, probability, current_node))

            for char, next_node in current_node.children.items():
                dfs(next_node, subword + char, list(collected_subwords))  # Create a copy of the list to avoid shared state

        dfs(node)
        best_subwords = sorted(best_subwords, key=lambda x: x[1], reverse=True)
        return [subword for subword, _, _ in best_subwords][:5] or [self.unk_token_id]


    def compute_failure_links(self):
        root = self.root
        root.failure_link = root  # Root's failure link points to itself
        queue = [root]

        while queue:
            current_node = queue.pop(0)

            for char, child_node in current_node.children.items():
                queue.append(child_node)

                # Follow failure link to find the longest suffix for the child_node
                failure_candidate = current_node.failure_link
                while failure_candidate != root and char not in failure_candidate.children:
                    failure_candidate = failure_candidate.failure_link
                child_node.failure_link = failure_candidate.children.get(char, root)


class SentencePiece:
    def __init__(self, unk_token="[UNK]"):
        self.trie = None
        self.vocab = {}
        self.unk_token = unk_token
        self.unk_token_id = 0  # Assign an ID for the unknown token

    def train(self, corpus, vocab_size=10000, model_type="unigram"):
        self.vocab = self._build_vocabulary(corpus, vocab_size, model_type)        
        self._precompute(self.vocab)
        return self.vocab

    def encode(self, text):
        """Encodes input text into subwords based on the trained model."""
        tokens = text.split() 
        encoded_tokens = []
        for token in tokens:
            subword_ids = self.trie.find_subwords(token)
            encoded_tokens.extend(subword_ids)
        return encoded_tokens

    def decode(self, subword_ids):
        """Reconstructs text from a sequence of subword IDs."""
        text = ""
        for subword_id in subword_ids:
            if subword_id != self.unk_token_id:
                text += self.vocab.get(subword_id, self.unk_token) 
            else:
                text += self.unk_token  
        return text

    def _build_vocabulary(self, corpus, vocab_size, model_type):
        """Constructs the SentencePiece vocabulary."""
        vocabulary = {}  # Use a dictionary for clarity
        if model_type == "unigram":
            all_tokens = []
            for text in corpus:
                tokens = self._unigram_tokenize(text)
                all_tokens.extend(tokens)
            vocabulary = self._build_unigram_vocab(all_tokens, vocab_size)
        elif model_type == "bpe":
            # Implement BPE vocabulary building if needed
            pass 
        else:
            raise ValueError("Invalid model_type. Use 'unigram' or 'bpe'")
        return vocabulary

    def _build_unigram_vocab(self, tokens, vocab_size):
        char_freqs = collections.Counter()
        for token in tokens:
            for char in token:
                char_freqs[char] += 1

        vocabulary = {char: idx for idx, (char, _) in enumerate(char_freqs.most_common(vocab_size))}
        return vocabulary

    def _unigram_tokenize(self, text):
        tokens = re.findall(r'\b\w+\b|[\s\.,!?;]', text)
        adjusted_tokens = []
        for token in tokens:
            if token in self.vocab:
                adjusted_tokens.append(token)
            else:
                adjusted_tokens.append(self.unk_token)
        return adjusted_tokens

    def _precompute(self, vocabulary):
        """Precomputes data structures for efficient subword segmentation."""
        self.trie = Trie(self.unk_token_id)
        for token_id, (token, frequency) in enumerate(vocabulary.items()):
            self.trie.insert(token, token_id, frequency)
        self.trie.compute_failure_links() 


class BPE:
    def __init__(self):
        self.vocab = None  # Will store vocabulary/frequency pairs
        self.num_merges = 10  # Default number of merge operations

    def train(self, corpus):
        """
        Trains the BPE model using the provided algorithm.
        Args:
            corpus: A text corpus represented as a list of strings.
        """
        self.vocab = self.init_vocab(corpus)
        for _ in range(self.num_merges):
            pairs = self.get_stats(self.vocab)
            if not pairs:  # Check if there are no more pairs to merge
                break
            best = max(pairs, key=pairs.get)
            self.vocab = self.merge_vocab(best, self.vocab)
            print("Merging: {}".format(best))  # Track most frequent pair in each iteration

        return self.vocab

    def init_vocab(self, corpus):
        """Creates initial vocabulary of words and their frequencies, tokenized at the character level."""
        vocab = collections.defaultdict(int)
        for text in corpus:
            words = text.strip().split()
            for word in words:
                # No longer splitting the word into characters with spaces
                vocab[word + " </w>"] += 1  # Marking end of word with </w>
        return vocab

    def preprocess_to_characters(self, word):
        return list(word)

    def encode(self, word):
        word_chars = self.preprocess_to_characters(word) 
        subwords = []

        while word_chars:  # Greedy encoding example
            for i in range(len(word_chars), 0, -1):
                subword = ''.join(word_chars[:i]) 
                if subword in self.vocab:
                    subwords.append(subword)
                    word_chars = word_chars[i:]
                    break 

        return subwords 

    def get_stats(self, vocab):
        """Gets frequency of character/subword pairs"""
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def merge_vocab(self, pair, vocab):
        """Replaces a frequent pair with a new symbol."""
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
        merged_vocab = {}
        for word, freq in vocab.items():
            new_word = p.sub(''.join(pair), word)
            merged_vocab[new_word] = merged_vocab.get(new_word, 0) + freq
        return merged_vocab


my_sp = SentencePiece()

# Sample corpus (pre-tokenization may need adjustment)
text_data = [
    "How transferable are features in deep neural networks?",
    "This is another sample."
]

model = my_sp.train(text_data) 
print(my_sp.vocab)
# Example encoding
result = my_sp.encode("transfer") 
print(result)  # This might output something like ['t', 'r', 'a', 'n', 'sf', 'er']

{'[': 0, 'U': 1, 'N': 2, 'K': 3, ']': 4}
[0]


# SP & BPE

In [90]:
import re
from collections import Counter, defaultdict

import json

class SimpleSentencePiece:
    def __init__(self, model_type="bpe", vocab_size=8000):
        self.vocab = {}
        self.id_to_subword = {}
        self.unk_token = "[UNK]"
        self.unk_token_id = 0
        self.vocab_size = vocab_size
        self.model = None if model_type == "bpe" else None  # Placeholder for future model type implementations
        self.model_type = model_type

    def train(self, text):
        if self.model_type == "bpe":
            self.model = BPE(num_merges=self.vocab_size, unk_token_id=self.unk_token_id) # Assuming you want to use vocab_size as num_merges
            self.model.train(text)
            self.vocab = self.model.vocab
            self.id_to_subword = {i: word for word, i in self.vocab.items()}
        else:
            raise NotImplementedError(f"Model type {self.model_type} not supported yet.")

        
    def encode(self, text):
        if not self.model:
            raise ValueError("Model has not been trained yet.")
        return self.model.encode(text)

    def decode(self, ids):
        if not self.id_to_subword:
            raise ValueError("Vocabulary is empty. Ensure the model is trained first.")
        # Reconstruct the text and remove the end-of-word marker '</w>'
        text = " ".join([self.id_to_subword.get(id_, self.unk_token) for id_ in ids])
        text = text.replace(" </w>", "").replace("</w>", " ").strip()
        return text
    
    def save_model(self, filepath):
        if self.model_type == "bpe":
            model_data = {
                'vocab': self.vocab,
                'merges': self.model.merges,
                'vocab_size': self.vocab_size,
                'model_type': self.model_type
            }
            with open(filepath, 'w') as f:
                json.dump(model_data, f)
                
    def load_model(self, filepath):
        with open(filepath, 'r') as f:
            model_data = json.load(f)
        self.vocab = model_data['vocab']
        self.id_to_subword = {i: word for word, i in self.vocab.items()}
        self.vocab_size = model_data.get('vocab_size', 8000)
        if model_data['model_type'] == "bpe":
            # Correctly initialize the BPE model with num_merges instead of vocab_size
            self.model = BPE(num_merges=self.vocab_size, unk_token_id=self.unk_token_id)
            self.model.merges = model_data['merges']
            # Since vocab is built based on merges, we need to rebuild it or ensure it's correctly loaded
            self.model.vocab = self.vocab
        else:
            raise NotImplementedError(f"Model type {model_data['model_type']} not supported yet.")




class BPE:
    def __init__(self, num_merges=100, unk_token_id=0):  # Accept unk_token_id parameter
        self.vocab = {}
        self.merges = []
        self.num_merges = num_merges
        self.unk_token_id = unk_token_id  # Store the unknown token ID

    def train(self, text):
        words = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
        vocab = collections.Counter(words)
        vocab = {word + '</w>': count for word, count in vocab.items()}
        
        for _ in range(self.num_merges):  # Use the num_merges from the instance variable
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best, vocab)
            self.merges.append(best)

        self.vocab = {word: i for i, word in enumerate(vocab.keys())}

    @staticmethod
    def get_stats(vocab):
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    @staticmethod
    def merge_vocab(pair, vocab):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in vocab:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = vocab[word]
        return v_out

    def encode(self, text):
        """Encode text into subwords using learned BPE merges."""
        encoded_tokens = []
        for word in re.findall(r'\w+|[^\w\s]', text, re.UNICODE):
            word += '</w>'
            subwords = [word]  # Start with the entire word as one subword
            for merge in self.merges:
                new_subwords = []
                for subword in subwords:
                    # If the merge is in subword, split it; otherwise, keep it as is
                    if ' '.join(merge) in subword:
                        new_subwords.extend(subword.replace(' '.join(merge), ''.join(merge)).split(' '))
                    else:
                        new_subwords.append(subword)
                subwords = new_subwords
            encoded_tokens.extend(subwords)
        return [self.vocab.get(token, self.unk_token_id) for token in encoded_tokens]
    
        # New method to save trained model
    def save_model(self, filepath):
        with open(filepath, 'w') as file:
            for merge in self.merges:
                file.write(' '.join(merge) + '\n')
    
    # New method to load trained model
    def load_model(self, filepath):
        self.merges = []
        with open(filepath, 'r') as file:
            for line in file:
                pair = tuple(line.strip().split(' '))
                self.merges.append(pair)



class WordPiece:
    def __init__(self, vocab):
        self.vocab = vocab
        self.unk_token = "[UNK]"
        self.root = self.build_trie(vocab)
        self.compute_failure_links(self.root)
        print("Trie built successfully.")

    # Add debug prints to build_trie to confirm structure
    def build_trie(self, vocab):
        root = TrieNode()
        for token in vocab:
            node = root
            for char in token:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_end = True
            node.token = token
        print("Trie Construction Completed Successfully")
        return root


    def compute_failure_links(self, root):
        queue = [root]
        while queue:
            current_node = queue.pop(0)
            for char, child_node in current_node.children.items():
                failure_node = current_node.failure_link
                while failure_node and char not in failure_node.children:
                    failure_node = failure_node.failure_link
                child_node.failure_link = failure_node.children[char] if failure_node else root
                queue.append(child_node)

    # Improved debug prints in tokenize method
    def tokenize(self, text):
        node = self.root
        tokens = []
        i = 0

        while i < len(text):
            char = text[i]
            if char == ' ':
                # Handle space as a delimiter
                print("Space encountered, resetting to root")
                node = self.root  # Reset to root
                i += 1  # Move to the next character
                continue

            if char not in node.children:
                print(f"Character '{char}' not found, adding [UNK] and resetting")
                tokens.append(self.unk_token)  # Add [UNK] for unmatched character
                node = self.root  # Reset to root
                i += 1  # Move to the next character
                continue  # Start fresh from the next character

            node = node.children[char]

            if node.is_end:
                print(f"Token found: '{node.token}'")
                tokens.append(node.token)  # Add the token found
                node = self.root  # Reset for the next token

            i += 1  # Move to the next character

        return tokens



Encoded Sentence: [0, 0, 28, 50, 26]
Decoded Sentence: SentencePiece  SentencePiece  subword  tokenization  .
Re-Encoded Sentence: [0, 0, 28, 50, 26]
Re-Decoded Sentence: SentencePiece  SentencePiece  subword  tokenization  .
Sample Vocabulary Check:
SentencePiece: 0
is: 1
an: 2
unsupervised: 3
text: 4
tokenizer: 5
and: 6
detokenizer: 7
mainly: 8
for: 9
Found 0 subtokens in vocabulary.
Trie Construction Completed Successfully
Trie built successfully.
Token found: 'SentencePiece'
Space encountered, resetting to root
Token found: 'provide'
Space encountered, resetting to root
Token found: 'subword'
Space encountered, resetting to root
Token found: 'to'
Character 'k' not found, adding [UNK] and resetting
Token found: 'e'
Character 'i' not found, adding [UNK] and resetting
Character 'z' not found, adding [UNK] and resetting
Token found: 'a'
Character 'i' not found, adding [UNK] and resetting
Character 'n' not found, adding [UNK] and resetting
Token found: '.'
Tokenized Sentence: ['Sentence

In [91]:

# Sample text for training the model
sample_text = """
SentencePiece is an unsupervised text tokenizer and detokenizer 
mainly for Neural Network-based text generation systems where the 
vocabulary size is predetermined prior to the neural model training. 
SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) 
and unigram language model) with the extension of direct training from 
raw sentences. Compared to other subword tokenization methods, 
SentencePiece can provide a better tokenization in some cases.
"""

# Number of merges for BPE; adjust based on your needs
num_merges = 100

# Initialize and train the SimpleSentencePiece model with BPE
ssp = SimpleSentencePiece(model_type="bpe", vocab_size=1000)
ssp.train(sample_text)

# Encode a sample sentence
encoded_sentence = ssp.encode("SentencePiece provides subword tokenization.")
print("Encoded Sentence:", encoded_sentence)

# Decode the encoded sentence
decoded_sentence = ssp.decode(encoded_sentence)
print("Decoded Sentence:", decoded_sentence)

# Save the trained model
ssp.save_model("ssp_model.json")

# Create a new instance and load the model
new_ssp = SimpleSentencePiece(model_type="bpe")
new_ssp.load_model("ssp_model.json")

# Re-encode and decode using the loaded model
re_encoded_sentence = new_ssp.encode("SentencePiece provides subword tokenization.")
re_decoded_sentence = new_ssp.decode(re_encoded_sentence)
print("Re-Encoded Sentence:", re_encoded_sentence)
print("Re-Decoded Sentence:", re_decoded_sentence)


# Assuming ssp is your SimpleSentencePiece instance
vocab = ssp.vocab  # This gets the vocabulary after BPE training

def adapt_vocab_for_wordpiece(ssp_vocab):
    adapted_vocab = {}
    for token, id_or_freq in ssp_vocab.items():
        # Check if a token is a continuation subword and not a standalone word
        # Since BPE might not mark subwords in a way WordPiece expects, we adapt based on our best approximation
        if not token.startswith(" ") and not token.endswith("</w>"):
            adapted_token = "##" + token.replace("</w>", "")  # Removing BPE's end-of-word marker and prepending "##"
        else:
            adapted_token = token.replace("</w>", "")  # Just remove the BPE's end-of-word marker for standalone words

        adapted_vocab[adapted_token] = id_or_freq
    return adapted_vocab


# Assuming ssp is your SimpleSentencePiece instance after BPE training
wordpiece_vocab = adapt_vocab_for_wordpiece(ssp.vocab)

# Debugging step to ensure vocabulary completeness
def debug_vocab(adapted_vocab):
    print("Sample Vocabulary Check:")
    # Iterate over the first 10 key-value pairs in the adapted vocabulary
    for i, (token, id_or_freq) in enumerate(adapted_vocab.items()):
        print(f"{token}: {id_or_freq}")
        if i >= 9:  # Stop after printing 10 entries
            break
    # Specifically check for subtokens if your tokenizer expects them
    subtokens = [token for token in adapted_vocab.keys() if token.startswith("##")]
    print(f"Found {len(subtokens)} subtokens in vocabulary.")


# Ensure wordpiece_vocab is a list of vocabulary tokens
debug_vocab(wordpiece_vocab)  # Call this after initializing wordpiece_vocab



# Initialize WordPiece with the adapted vocabulary
wordpiece = WordPiece(wordpiece_vocab)

# Tokenize a sample sentence using WordPiece
sample_sentence = "SentencePiece provides subword tokenization."
tokenized_sentence = wordpiece.tokenize(sample_sentence)
print("Tokenized Sentence:", tokenized_sentence)


# Creating a dictionary of test sentences as outlined in the instructions

test_sentences = {
    "known_words": "SentencePiece is an unsupervised tokenizer.",
    "subwords": "Subword tokenization improves neural model performance.",
    "mixed_sentence": "Tokenizing with SentencePiece and BPE provides flexibility.",
    "case_sensitivity": "sentencepiece provides SubWord Tokenization.",
    "punctuation_handling": "Tokenization, especially for NLP, is crucial.",
    "complex_subword": "Neural networks benefit from subword-level processing."
}

# Display the dictionary for confirmation
test_sentences

for label, sentence in test_sentences.items():
    print(f"Testing '{label}': {sentence}")
    tokenized = wordpiece.tokenize(sentence)
    print(f"Tokenized: {tokenized}\n")

Testing 'known_words': SentencePiece is an unsupervised tokenizer.
Token found: 'SentencePiece'
Space encountered, resetting to root
Token found: 'is'
Space encountered, resetting to root
Token found: 'a'
Space encountered, resetting to root
Token found: 'unsupervised'
Space encountered, resetting to root
Token found: 'to'
Character 'k' not found, adding [UNK] and resetting
Token found: 'e'
Character 'i' not found, adding [UNK] and resetting
Character 'z' not found, adding [UNK] and resetting
Token found: 'e'
Character '.' not found, adding [UNK] and resetting
Tokenized: ['SentencePiece', 'is', 'a', 'unsupervised', 'to', '[UNK]', 'e', '[UNK]', '[UNK]', 'e', '[UNK]']

Testing 'subwords': Subword tokenization improves neural model performance.
Character 'u' not found, adding [UNK] and resetting
Character 'w' not found, adding [UNK] and resetting
Character 'r' not found, adding [UNK] and resetting
Space encountered, resetting to root
Token found: 'to'
Character 'k' not found, adding [UNK]

# probsolve

In [94]:
import re
from collections import Counter, defaultdict

import json
class SimpleSentencePiece:
    def __init__(self, model_type="bpe", vocab_size=8000):
        self.vocab = {}
        self.id_to_subword = {}
        self.unk_token = "[UNK]"
        self.unk_token_id = 0
        self.vocab_size = vocab_size
        self.model = None if model_type == "bpe" else None
        self.model_type = model_type

    def train(self, text):
        if self.model_type == "bpe":
            self.model = BPE(num_merges=self.vocab_size, unk_token_id=self.unk_token_id)
            self.model.train(text)
            self.vocab = self.model.vocab
            self.id_to_subword = {i: word for word, i in self.vocab.items()}
        else:
            raise NotImplementedError(f"Model type {self.model_type} not supported yet.")

    def encode(self, text):
        text = self.preprocess_text(text)  # Preprocess text before encoding
        if not self.model:
            raise ValueError("Model has not been trained yet.")
        return self.model.encode(text)

    def decode(self, ids):
        if not self.id_to_subword:
            raise ValueError("Vocabulary is empty. Ensure the model is trained first.")
        text = " ".join([self.id_to_subword.get(id_, self.unk_token) for id_ in ids])
        text = text.replace(" </w>", "").replace("</w>", " ").strip()
        return text

    def preprocess_text(self, text):
        # Convert text to lowercase to ensure case insensitivity
        text = text.lower()
        # Optionally, handle punctuation by adding spaces around it for better tokenization
        text = re.sub(r'([.,!?()])', r' \1 ', text)
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        # Trim leading and trailing spaces
        text = text.strip()
        return text
    
    def save_model(self, filepath):
        model_data = {
            'vocab': self.vocab,
            'id_to_subword': self.id_to_subword,
            'model_type': self.model_type,
            'vocab_size': self.vocab_size,
            # Potentially include other relevant attributes
        }
        # Save the high-level tokenizer settings
        with open(filepath, 'w') as f:
            json.dump(model_data, f)
        
        # Now save the BPE model specifically
        if self.model_type == "bpe" and self.model:
            self.model.save_model(filepath + "_bpe")

    def load_model(self, filepath):
        with open(filepath, 'r') as f:
            model_data = json.load(f)
        
        self.vocab = model_data['vocab']
        self.id_to_subword = model_data['id_to_subword']
        self.model_type = model_data['model_type']
        self.vocab_size = model_data['vocab_size']
        
        # Assuming model_type is still "bpe", we now load the BPE model
        if self.model_type == "bpe":
            self.model = BPE(self.vocab_size, self.unk_token_id)
            self.model.load_model(filepath + "_bpe")




class BPE:
    def __init__(self, num_merges=100, unk_token_id=0):  # Accept unk_token_id parameter
        self.vocab = {}
        self.merges = []
        self.num_merges = num_merges
        self.unk_token_id = unk_token_id  # Store the unknown token ID

    def train(self, text):
        words = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
        vocab = collections.Counter(words)
        vocab = {word + '</w>': count for word, count in vocab.items()}
        
        for _ in range(self.num_merges):  # Use the num_merges from the instance variable
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best, vocab)
            self.merges.append(best)

        self.vocab = {word: i for i, word in enumerate(vocab.keys())}

    @staticmethod
    def get_stats(vocab):
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    @staticmethod
    def merge_vocab(pair, vocab):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in vocab:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = vocab[word]
        return v_out

    def encode(self, text):
        """Encode text into subwords using learned BPE merges."""
        encoded_tokens = []
        for word in re.findall(r'\w+|[^\w\s]', text, re.UNICODE):
            word += '</w>'
            subwords = [word]  # Start with the entire word as one subword
            for merge in self.merges:
                new_subwords = []
                for subword in subwords:
                    # If the merge is in subword, split it; otherwise, keep it as is
                    if ' '.join(merge) in subword:
                        new_subwords.extend(subword.replace(' '.join(merge), ''.join(merge)).split(' '))
                    else:
                        new_subwords.append(subword)
                subwords = new_subwords
            encoded_tokens.extend(subwords)
        return [self.vocab.get(token, self.unk_token_id) for token in encoded_tokens]
    
        # New method to save trained model
    def save_model(self, filepath):
        bpe_data = {
            'merges': self.merges,
            'vocab': self.vocab,
            'num_merges': self.num_merges,
            # Include other attributes as needed
        }
        with open(filepath, 'w') as f:
            json.dump(bpe_data, f)

    def load_model(self, filepath):
        with open(filepath, 'r') as f:
            bpe_data = json.load(f)
        
        self.merges = bpe_data['merges']
        self.vocab = bpe_data['vocab']
        self.num_merges = bpe_data['num_merges']



class WordPiece:
    def __init__(self, vocab):
        self.vocab = vocab
        self.unk_token = "[UNK]"
        self.root = self.build_trie(vocab)
        self.compute_failure_links(self.root)
        print("Trie built successfully.")

    # Add debug prints to build_trie to confirm structure
    def build_trie(self, vocab):
        root = TrieNode()
        for token in vocab:
            node = root
            for char in token:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_end = True
            node.token = token
        print("Trie Construction Completed Successfully")
        return root


    def compute_failure_links(self, root):
        queue = [root]
        while queue:
            current_node = queue.pop(0)
            for char, child_node in current_node.children.items():
                failure_node = current_node.failure_link
                while failure_node and char not in failure_node.children:
                    failure_node = failure_node.failure_link
                child_node.failure_link = failure_node.children[char] if failure_node else root
                queue.append(child_node)

    # Improved debug prints in tokenize method
    def tokenize(self, text):
        # Preprocess input text
        text = self.preprocess_text(text)
        node = self.root
        tokens = []
        i = 0

        while i < len(text):
            char = text[i]
            if char == ' ':
                node = self.root
                i += 1
                continue

            if char not in node.children:
                if node != self.root:
                    tokens.append(node.token)  # Add the longest token found
                    node = self.root  # Reset to root
                    continue
                else:
                    tokens.append(self.unk_token)
                    i += 1
                    continue

            node = node.children[char]
            if node.is_end:
                if i + 1 == len(text) or text[i + 1] == ' ':
                    tokens.append(node.token)
                    node = self.root

            i += 1

        return tokens
    
    def preprocess_text(self, text):
        # Convert text to lowercase to ensure case insensitivity
        text = text.lower()

        # Optionally, handle punctuation by adding spaces around it for better tokenization
        # This depends on how your vocabulary handles punctuation
        text = re.sub(r'([.,!?()])', r' \1 ', text)

        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)

        # Trim leading and trailing spaces
        text = text.strip()

        return text


# Sample text for training the model
sample_text = """
SentencePiece is an unsupervised text tokenizer and detokenizer 
mainly for Neural Network-based text generation systems where the 
vocabulary size is predetermined prior to the neural model training. 
SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) 
and unigram language model) with the extension of direct training from 
raw sentences. Compared to other subword tokenization methods, 
SentencePiece can provide a better tokenization in some cases.
"""

# Number of merges for BPE; adjust based on your needs
num_merges = 100

# Initialize and train the SimpleSentencePiece model with BPE
ssp = SimpleSentencePiece(model_type="bpe", vocab_size=1000)
ssp.train(sample_text)

# Encode and decode a sample sentence
encoded_sentence = ssp.encode("SentencePiece provides subword tokenization.")
decoded_sentence = ssp.decode(encoded_sentence)
print("Encoded Sentence:", encoded_sentence)
print("Decoded Sentence:", decoded_sentence)

# Save the trained model
ssp.save_model("ssp_model.json")

# Create a new instance and load the model
new_ssp = SimpleSentencePiece(model_type="bpe")
new_ssp.load_model("ssp_model.json")

# Re-encode and decode using the loaded model
re_encoded_sentence = new_ssp.encode("SentencePiece provides subword tokenization.")
re_decoded_sentence = new_ssp.decode(re_encoded_sentence)
print("Re-Encoded Sentence:", re_encoded_sentence)
print("Re-Decoded Sentence:", re_decoded_sentence)


# Assuming ssp is your SimpleSentencePiece instance
vocab = ssp.vocab  # This gets the vocabulary after BPE training

def adapt_vocab_for_wordpiece(ssp_vocab):
    adapted_vocab = {}
    for token, id_or_freq in ssp_vocab.items():
        # Check if a token is a continuation subword and not a standalone word
        # Since BPE might not mark subwords in a way WordPiece expects, we adapt based on our best approximation
        if not token.startswith(" ") and not token.endswith("</w>"):
            adapted_token = "##" + token.replace("</w>", "")  # Removing BPE's end-of-word marker and prepending "##"
        else:
            adapted_token = token.replace("</w>", "")  # Just remove the BPE's end-of-word marker for standalone words

        adapted_vocab[adapted_token] = id_or_freq
    return adapted_vocab


# Assuming ssp is your SimpleSentencePiece instance after BPE training
wordpiece_vocab = adapt_vocab_for_wordpiece(ssp.vocab)

# Debugging step to ensure vocabulary completeness
def debug_vocab(adapted_vocab):
    print("Sample Vocabulary Check:")
    # Iterate over the first 10 key-value pairs in the adapted vocabulary
    for i, (token, id_or_freq) in enumerate(adapted_vocab.items()):
        print(f"{token}: {id_or_freq}")
        if i >= 9:  # Stop after printing 10 entries
            break
    # Specifically check for subtokens if your tokenizer expects them
    subtokens = [token for token in adapted_vocab.keys() if token.startswith("##")]
    print(f"Found {len(subtokens)} subtokens in vocabulary.")


# Ensure wordpiece_vocab is a list of vocabulary tokens
debug_vocab(wordpiece_vocab)  # Call this after initializing wordpiece_vocab



# Initialize WordPiece with the adapted vocabulary
wordpiece = WordPiece(wordpiece_vocab)

# Tokenize a sample sentence using WordPiece
sample_sentence = "SentencePiece provides subword tokenization."
tokenized_sentence = wordpiece.tokenize(sample_sentence)
print("Tokenized Sentence:", tokenized_sentence)


# Creating a dictionary of test sentences as outlined in the instructions

test_sentences = {
    "known_words": "SentencePiece is an unsupervised tokenizer.",
    "subwords": "Subword tokenization improves neural model performance.",
    "mixed_sentence": "Tokenizing with SentencePiece and BPE provides flexibility.",
    "case_sensitivity": "sentencepiece provides SubWord Tokenization.",
    "punctuation_handling": "Tokenization, especially for NLP, is crucial.",
    "complex_subword": "Neural networks benefit from subword-level processing."
}

# Display the dictionary for confirmation
test_sentences

for label, sentence in test_sentences.items():
    print(f"Testing '{label}': {sentence}")
    tokenized = wordpiece.tokenize(sentence)
    print(f"Tokenized: {tokenized}\n")





Encoded Sentence: [0, 0, 28, 50, 26]
Decoded Sentence: SentencePiece  SentencePiece  subword  tokenization  .
Re-Encoded Sentence: [0, 0, 28, 50, 26]
Re-Decoded Sentence: [UNK] [UNK] [UNK] [UNK] [UNK]
Sample Vocabulary Check:
SentencePiece: 0
is: 1
an: 2
unsupervised: 3
text: 4
tokenizer: 5
and: 6
detokenizer: 7
mainly: 8
for: 9
Found 0 subtokens in vocabulary.
Trie Construction Completed Successfully
Trie built successfully.
Tokenized Sentence: [None, None, None, 'e', None, 'e', 'provide', 'subword', 'tokenization', '.']
Testing 'known_words': SentencePiece is an unsupervised tokenizer.
Tokenized: [None, None, None, 'e', None, 'e', 'is', 'an', 'unsupervised', 'tokenizer', '.']

Testing 'subwords': Subword tokenization improves neural model performance.
Tokenized: ['subword', 'tokenization', None, None, None, None, 'e', 'neural', 'model', None, 'e', None, 'for', None, None, None, 'e', '.']

Testing 'mixed_sentence': Tokenizing with SentencePiece and BPE provides flexibility.
Tokenized:

In [105]:
# Test on larger text
import re
from collections import Counter, defaultdict

import json
class SimpleSentencePiece:
    def __init__(self, model_type="bpe", vocab_size=8000):
        self.vocab = {}
        self.id_to_subword = {}
        self.unk_token = "[UNK]"
        self.unk_token_id = 0
        self.vocab_size = vocab_size
        self.model = None if model_type == "bpe" else None
        self.model_type = model_type

    def train(self, text):
        if self.model_type == "bpe":
            self.model = BPE(num_merges=self.vocab_size, unk_token_id=self.unk_token_id)
            self.model.train(text)
            self.vocab = self.model.vocab
            self.id_to_subword = {i: word for word, i in self.vocab.items()}
        else:
            raise NotImplementedError(f"Model type {self.model_type} not supported yet.")

    def encode(self, text):
        text = self.preprocess_text(text)  # Preprocess text before encoding
        if not self.model:
            raise ValueError("Model has not been trained yet.")
        return self.model.encode(text)

    def decode(self, ids):
        if not self.id_to_subword:
            raise ValueError("Vocabulary is empty. Ensure the model is trained first.")
        text = " ".join([self.id_to_subword.get(id_, self.unk_token) for id_ in ids])
        text = text.replace(" </w>", "").replace("</w>", " ").strip()
        return text

    def preprocess_text(self, text):
        # Convert text to lowercase to ensure case insensitivity
        text = text.lower()
        # Optionally, handle punctuation by adding spaces around it for better tokenization
        text = re.sub(r'([.,!?()])', r' \1 ', text)
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        # Trim leading and trailing spaces
        text = text.strip()
        return text
    
    def save_model(self, filepath):
        model_data = {
            'vocab': self.vocab,
            'id_to_subword': self.id_to_subword,
            'model_type': self.model_type,
            'vocab_size': self.vocab_size,
            # Potentially include other relevant attributes
        }
        # Save the high-level tokenizer settings
        with open(filepath, 'w') as f:
            json.dump(model_data, f)
        
        # Now save the BPE model specifically
        if self.model_type == "bpe" and self.model:
            self.model.save_model(filepath + "_bpe")

    def load_model(self, filepath):
        with open(filepath, 'r') as f:
            model_data = json.load(f)
        
        self.vocab = model_data['vocab']
        self.id_to_subword = model_data['id_to_subword']
        self.model_type = model_data['model_type']
        self.vocab_size = model_data['vocab_size']
        
        # Assuming model_type is still "bpe", we now load the BPE model
        if self.model_type == "bpe":
            self.model = BPE(self.vocab_size, self.unk_token_id)
            self.model.load_model(filepath + "_bpe")

class BPE:
    def __init__(self, num_merges=100, unk_token_id=0):  # Accept unk_token_id parameter
        self.vocab = {}
        self.merges = []
        self.num_merges = num_merges
        self.unk_token_id = unk_token_id  # Store the unknown token ID

    def train(self, text):
        words = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
        vocab = collections.Counter(words)
        vocab = {word + '</w>': count for word, count in vocab.items()}
        
        for _ in range(self.num_merges):  # Use the num_merges from the instance variable
            pairs = self.get_stats(vocab)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best, vocab)
            self.merges.append(best)

        self.vocab = {word: i for i, word in enumerate(vocab.keys())}

    @staticmethod
    def get_stats(vocab):
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    @staticmethod
    def merge_vocab(pair, vocab):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in vocab:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = vocab[word]
        return v_out

    def encode(self, text):
        """Encode text into subwords using learned BPE merges."""
        encoded_tokens = []
        for word in re.findall(r'\w+|[^\w\s]', text, re.UNICODE):
            word += '</w>'
            subwords = [word]  # Start with the entire word as one subword
            for merge in self.merges:
                new_subwords = []
                for subword in subwords:
                    # If the merge is in subword, split it; otherwise, keep it as is
                    if ' '.join(merge) in subword:
                        new_subwords.extend(subword.replace(' '.join(merge), ''.join(merge)).split(' '))
                    else:
                        new_subwords.append(subword)
                subwords = new_subwords
            encoded_tokens.extend(subwords)
        return [self.vocab.get(token, self.unk_token_id) for token in encoded_tokens]
    
        # New method to save trained model
    def save_model(self, filepath):
        bpe_data = {
            'merges': self.merges,
            'vocab': self.vocab,
            'num_merges': self.num_merges,
            # Include other attributes as needed
        }
        with open(filepath, 'w') as f:
            json.dump(bpe_data, f)

    def load_model(self, filepath):
        with open(filepath, 'r') as f:
            bpe_data = json.load(f)
        
        self.merges = bpe_data['merges']
        self.vocab = bpe_data['vocab']
        self.num_merges = bpe_data['num_merges']

class WordPiece:
    def __init__(self, vocab):
        self.vocab = vocab
        self.unk_token = "[UNK]"
        self.root = self.build_trie(vocab)
        self.compute_failure_links(self.root)
        print("Trie built successfully.")

    # Add debug prints to build_trie to confirm structure
    def build_trie(self, vocab):
        root = TrieNode()
        for token in vocab:
            node = root
            for char in token:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_end = True
            node.token = token
        print("Trie Construction Completed Successfully")
        return root


    def compute_failure_links(self, root):
        queue = [root]
        while queue:
            current_node = queue.pop(0)
            for char, child_node in current_node.children.items():
                failure_node = current_node.failure_link
                while failure_node and char not in failure_node.children:
                    failure_node = failure_node.failure_link
                child_node.failure_link = failure_node.children[char] if failure_node else root
                queue.append(child_node)

    # Improved debug prints in tokenize method
    def tokenize(self, text):
        # Preprocess input text
        text = self.preprocess_text(text)
        node = self.root
        tokens = []
        i = 0

        while i < len(text):
            char = text[i]
            if char == ' ':
                node = self.root
                i += 1
                continue

            if char not in node.children:
                if node != self.root:
                    tokens.append(node.token)  # Add the longest token found
                    node = self.root  # Reset to root
                    continue
                else:
                    tokens.append(self.unk_token)
                    i += 1
                    continue

            node = node.children[char]
            if node.is_end:
                if i + 1 == len(text) or text[i + 1] == ' ':
                    tokens.append(node.token)
                    node = self.root

            i += 1

        return tokens
    
    def preprocess_text(self, text):
        # Convert text to lowercase to ensure case insensitivity
        text = text.lower()

        # Optionally, handle punctuation by adding spaces around it for better tokenization
        # This depends on how your vocabulary handles punctuation
        text = re.sub(r'([.,!?()])', r' \1 ', text)

        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)

        # Trim leading and trailing spaces
        text = text.strip()

        return text


def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = [line.strip() for line in file.readlines()]
    return texts

texts = load_corpus("D:\\EXPERT_WEIGHTS\\sample.txt")

num_merges = 100

# Initialize and train the SimpleSentencePiece model with BPE
ssp = SimpleSentencePiece(model_type="bpe", vocab_size=1000)
ssp.train('\n'.join(texts))  # Train the model on the entire dataset

# Test the model on a subset or the entire dataset
for i, text in enumerate(texts[:100]):  # Example: test on the first 100 texts
    encoded = ssp.encode(text)
    decoded = ssp.decode(encoded)
    print(f"Original: {text}")
    print(f"Decoded: {decoded}\n")

# Save the trained model
ssp.save_model("ssp_model.json")

# Create a new instance and load the model
new_ssp = SimpleSentencePiece(model_type="bpe")
new_ssp.load_model("ssp_model.json")

# New Model
for i, text in enumerate(texts[:100]):  # Example: test on the first 100 texts
    re_encoded = ssp.encode(text)
    re_decoded = ssp.decode(re_encoded)
    print(f"Original: {text}")
    print(f"Decoded: {re_decoded}\n")


# Assuming ssp is your SimpleSentencePiece instance
vocab = ssp.vocab  # This gets the vocabulary after BPE training
new_vocab = new_ssp.vocab  # This gets the vocabulary after BPE training

print(f"vocab ; {len(vocab)} , new_vocab: {len(new_vocab)}")

def adapt_vocab_for_wordpiece(ssp_vocab):
    adapted_vocab = {}
    for token, id_or_freq in ssp_vocab.items():
        # Check if a token is a continuation subword and not a standalone word
        # Since BPE might not mark subwords in a way WordPiece expects, we adapt based on our best approximation
        if not token.startswith(" ") and not token.endswith("</w>"):
            adapted_token = "##" + token.replace("</w>", "")  # Removing BPE's end-of-word marker and prepending "##"
        else:
            adapted_token = token.replace("</w>", "")  # Just remove the BPE's end-of-word marker for standalone words

        adapted_vocab[adapted_token] = id_or_freq
    return adapted_vocab


# Assuming ssp is your SimpleSentencePiece instance after BPE training
wordpiece_vocab = adapt_vocab_for_wordpiece(ssp.vocab)

# Debugging step to ensure vocabulary completeness
def debug_vocab(adapted_vocab):
    print("Sample Vocabulary Check:")
    # Iterate over the first 10 key-value pairs in the adapted vocabulary
    for i, (token, id_or_freq) in enumerate(adapted_vocab.items()):
        print(f"{token}: {id_or_freq}")
        if i >= 9:  # Stop after printing 10 entries
            break
    # Specifically check for subtokens if your tokenizer expects them
    subtokens = [token for token in adapted_vocab.keys() if token.startswith("##")]
    print(f"Found {len(subtokens)} subtokens in vocabulary.")


# Ensure wordpiece_vocab is a list of vocabulary tokens
debug_vocab(wordpiece_vocab)  # Call this after initializing wordpiece_vocab

# Initialize WordPiece with the adapted vocabulary
wordpiece = WordPiece(wordpiece_vocab)

for i, text in enumerate(texts[:100]):
    print(f"Testing ': {text}")
    tokenized = wordpiece.tokenize(text)
    print(f"Tokenized: {tokenized}\n")

Original: Fast WordPiece Tokenization
Decoded: Fast  Fast  tokenization

Original: Xinying Song† Alex Salcianu† Yang Song‡∗ Dave Dopson† Denny Zhou†
Decoded: Fast  Fast  †  Fast  salcianu  †  Fast  Fast  ‡  ∗  Fast  Fast  †  Fast  Fast  †

Original: †Google Research, Mountain View, CA
Decoded: †  google  Fast  ,  mountain  Fast  ,  Fast

Original: †{xysong,salcianu,ddopson,dennyzhou}@google.com
Decoded: †  {  xysong  ,  salcianu  ,  ddopson  ,  dennyzhou  }  @  google  .  com

Original: ‡Kuaishou Technology, Beijing, China
Decoded: ‡  kuaishou  Fast  ,  Fast  ,  Fast

Original: ‡yangsong@kuaishou.com
Decoded: ‡  yangsong  @  kuaishou  .  com

Original: Abstract
Decoded: Fast

Original: Tokenization is a fundamental preprocessing
Decoded: tokenization  is  a  fundamental  preprocessing

Original: step for almost all NLP tasks. In this paper,
Decoded: step  for  almost  all  Fast  tasks  .  in  this  paper  ,

Original: we propose efficient algorithms for the WordPiece tokenization used 

# Test the WordPiece